# Introduction
This is taking https://www.kaggle.com/lisphilar's notebook and optimizing for parameters and trying things to validate the predictions. 

### Major update
 * 13Feb2020: SIR model
 * 15Feb2020: SIR-D model
 * 22Feb2020: SIR-F model
 * 23Feb2020: Changed the dataset from 2019_ncov_data.csv to covid_19_data.csv
 * 23Feb2020: $\tau$ was fixed as "1 day" because the time format of ObservationDate is MM/DD/YYYY
 * 23Feb2020: SIR-F model with other countries
 * 23Feb2020: How to minimize the damage (Change of parameter, Vacctination)
 * 24Feb2020: Use $\tau$ again
 * 01Mar2020: $\tau$ definition was changed. $1\leq \tau \mathrm{[hour]} \leq 24$ $\to$ $1\leq \tau \mathrm{[min]} \leq 1440$ 
 * 01Mar2020: Added "Test of hyperparameter optimization using example data" in SIR model section
 * 02Mar2020: Analysis of Linelist (estimation of Effective contact/Recovery/Death rate using case reports)
 * 03Mar2020: Trend analysis
 * 03Mar2020: Update estimator error function; Exponential Weighted Moving Average (span=14days) of |observed - estimated|
 * 04Mar2020: "Analysis of Linelist" was moved to [EDA of linelist](https://www.kaggle.com/lisphilar/eda-of-linelist?scriptVersionId=29640733#Remarks)
 * 04Mar2020: Data in Hubei and China will be analysed in another notebook. Please refer to [Data in China with SIR model](https://www.kaggle.com/lisphilar/data-in-china-with-sir-model?scriptVersionId=29646940).
 * 06Mar2020: Random seed was fixed as 2019
 * 06Mar2020: Update estimator error function; Weighted Average of |Exponential Weighted Moving Average (span=14days) of observed - estimated|
 * 07Mar2020: Update estimator error function; Total population $\times$ Wighted Average of |observed - estimated| with step number t
 * 07Mar2020: Priorities of variables in estimator error function was set as $(x, y, z, w) = (1, 10, 10, 1)$ in SIR-F model.
 * 09Mar2020: Update estimator error function; $(\mathrm{Population})^2$ $\times$ (Wighted Average of |observed - estimated|/[observed $\times$ Population + 1] with step number t)
 * 09Mar2020: Priorities of variables in estimator error function were set as $(x, y, z, w) = (1, 10, 10, 2)$ in SIR-F model.
 * 11Mar2020: Update model.param_dict(); each parametor range was limited to 30%-70% quantiles of the estimated values ($\frac{\mathrm{d}z}{\mathrm{d}t}\left(\frac{1}{y}\right)$ for $\sigma$) of training dataset.
 * 12Mar2020: Update model.param_dict(); each parameter range was limited to 5%-95% quantiles
 * 12Mar2020: Detailed scenario analysis. Thank you, Marco Ferrante!
 * 13Mar2020: Update model.param_dict(); each parameter range was limited to 0%-100% quantiles
 * 13Mar2020: Update "Detailed scenario analysis" > "Real factors of effective contact rate $\beta$"
 * 14Mar2020: Update model.param_dict(); rho/sigma range was limited to 30%-70% quantiles of their estimated values
 * 14Mar2020: Applied trend analysis on country level data to use only a part of records for estimation
 * 14Mar2020: Recovered without confirmation was added to "Real factors of effective contact rate $\beta$"
 * 14Mar2020: Resource of total population values was changed to [PopulationPyramid.net WORLD 2020](https://www.populationpyramid.net/world/2020/)
 * 15Mar2020: Merge "How to minimize the damage (Change of parameter, Vacctination)" with "Scenario analysis" section
 * 15Mar2020: Update Estimator, int to np.int. Thank you Enrico Papalini!
 * 15Mar2020: Update Estimator, some parameters can be fixed. Some of SIR parameters can be applied to SIR-F model.
 * 17Mar2020: The number of exposed cases and waiting cases
 * 17Mar2020: Update Scenario analysis
 * 18Mar2020: Scenario analysis in Italy
 * 19Mar2020: Estimation of new drugs effect in "Scenario analysis in Italy" section
 
 <!--* 10Mar2020: In Estimator.objective(), apply adjusted Exponential Moving Average (span=7days) on the training data-->
 
 <!--* 08Mar2020: ODE solver method was changed from RK45 to Radau (Implicit Runge-Kutta method of the Radau IIA family of order 5) because max value of $\tau$ is 1440[min] and the number of iterations tends to small. The numerical simulation system seems stiff equation. Please refer to [scipy.integrate.solve_ivp guide](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).-->

In [ ]:
from datetime import datetime
time_format = "%d%b%Y %H:%M"
datetime.now().strftime(time_format)

# Arrangement of dataset

## Package

In [ ]:
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import os
from pprint import pprint
import warnings
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import ScalarFormatter
%matplotlib inline
import numpy as np
import optuna
optuna.logging.disable_default_handler()
import pandas as pd
pd.plotting.register_matplotlib_converters()
import seaborn as sns
from scipy.integrate import solve_ivp

In [ ]:
np.random.seed(2019)
os.environ["PYTHONHASHSEED"] = "2019"

In [ ]:
plt.style.use("seaborn-ticks")
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["font.size"] = 11.0
plt.rcParams["figure.figsize"] = (9, 6)

## Total population and population pyramid
**Total population values and population pyramid data of each country in 2020 was retrieved in CSV format from [PopulationPyramid.net](https://www.populationpyramid.net/) licenced under [Creative Commons license CC BY 3.0](https://creativecommons.org/licenses/by/3.0/igo/).** Data was transformed for ease to use in this notebook.

In [ ]:
_age_bins = [
    "0-4", "5-9", "10-14", "15-19", "20-24", "25-29",
    "30-34", "35-39", "40-44", "45-49", "50-54", "55-59",
    "60-64", "65-69", "70-74", "75-79", "80-84", "85-89",
    "90-94", "95-99", "100+"
]
_pyramid_df = pd.DataFrame({"Age_bin": _age_bins})

In [ ]:
# Global (WORLD)
_name = "Global"
_male = [
    349432556,
    342927576,
    331497486,
    316642222,
    308286775,
    306059387,
    309236984,
    276447037,
    249389688,
    241232876,
    222609691,
    192215395,
    157180267,
    128939392,
    87185982,
    54754941,
    33648953,
    15756942,
    5327866,
    1077791,
    124144
]
_female = [
    328509234,
    321511867,
    309769906,
    295553758,
    289100903,
    288632766,
    296293748,
    268371754,
    244399176,
    238133281,
    223162982,
    195633743,
    164961323,
    140704320,
    101491347,
    69026831,
    48281201,
    26429329,
    11352182,
    3055845,
    449279
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_name = "China"
_male = [
    44456332,
    46320144,
    45349923,
    44103122,
    46273865,
    51522843,
    66443228,
    51345507,
    49289359,
    61173349,
    62348020,
    49958045,
    38917285,
    36526788,
    21425163,
    12207276,
    6883629,
    2843084,
    731228,
    116377,
    12773
]
_female = [
    39476105,
    40415039,
    38912828,
    38238737,
    40884302,
    46466160,
    62295742,
    48745948,
    46984787,
    58664268,
    61097362,
    48782446,
    38596854,
    37622978,
    23524526,
    14337340,
    9297788,
    4738693,
    1573796,
    358816,
    61919
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_pyramid_df["Except China"] = _pyramid_df["Global"] - _pyramid_df["China"]

In [ ]:
_name = "Japan"
_male = [
    2453834,
    2773482,
    2856888,
    2926787,
    3075580,
    3151556,
    3475526,
    3918085,
    4307199,
    5088697,
    4364792,
    3964557,
    3733454,
    4095950,
    4312462,
    3160764,
    2209841,
    1282793,
    495632,
    95394,
    9772
]
_female = [
    2324647,
    2628006,
    2707638,
    2775858,
    2921297,
    2998892,
    3314219,
    3747586,
    4161221,
    4915957,
    4293819,
    3918347,
    3762668,
    4283163,
    4814856,
    3897293,
    3129208,
    2347551,
    1290191,
    422131,
    68864
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_name = "Italy"
_male = [
    1197289,
    1374731,
    1470174,
    1484455,
    1526577,
    1625230,
    1702976,
    1824273,
    2092329,
    2401070,
    2420466,
    2274884,
    1903045,
    1679600,
    1586760,
    1182323,
    958360,
    504059,
    186608,
    39461,
    3055
]
_female = [
    1127405,
    1295570,
    1387183,
    1391636,
    1415929,
    1535700,
    1662536,
    1808649,
    2096655,
    2431950,
    2487780,
    2384062,
    2050522,
    1851695,
    1805038,
    1454787,
    1344033,
    893202,
    453131,
    133178,
    13462
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
# South Korea (Republic of Korea)
_name = "South Korea"
_male = [
    974300,
    1158751,
    1174844,
    1289269,
    1683129,
    1870859,
    1733487,
    1974613,
    2015813,
    2186422,
    2168485,
    2082574,
    1855904,
    1287861,
    929821,
    664900,
    403886,
    160921,
    41739,
    7689,
    587
]
_female = [
    922711,
    1098051,
    1101938,
    1187207,
    1533115,
    1629191,
    1548938,
    1822801,
    1909121,
    2107488,
    2147031,
    2078609,
    1918662,
    1391279,
    1068270,
    897655,
    679943,
    379182,
    143170,
    35207,
    3760
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_name = "Iran"
_male = [
    3913297,
    3566704,
    3185825,
    2829745,
    2801162,
    3382446,
    4211840,
    4120404,
    3270956,
    2608048,
    2321587,
    1826123,
    1559797,
    1128726,
    709933,
    471441,
    306847,
    162121,
    27495,
    3670,
    238
]
_female = [
    3724262,
    3361593,
    3032120,
    2700238,
    2755066,
    3404233,
    4254862,
    4152339,
    3220365,
    2556247,
    2296892,
    1850729,
    1572477,
    1135905,
    738068,
    435165,
    244765,
    115360,
    29181,
    4400,
    280
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_name = "UK"
_male = [
    2009363,
    2108550,
    2022370,
    1880611,
    2072674,
    2275138,
    2361054,
    2279836,
    2148253,
    2128343,
    2281421,
    2232388,
    1919839,
    1647391,
    1624635,
    1137438,
    766956,
    438663,
    169952,
    34524,
    3016
]
_female = [
    1915127,
    2011016,
    1933970,
    1805522,
    2001966,
    2208929,
    2345774,
    2308360,
    2159877,
    2167778,
    2353119,
    2306537,
    1985177,
    1734370,
    1763853,
    1304709,
    969611,
    638892,
    320625,
    95559,
    12818
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
_name = "US"
_male = [
    10055063,
    10246393,
    10777513,
    10834321,
    11322732,
    12144455,
    11702514,
    10858871,
    10118582,
    9969099,
    10319535,
    10702065,
    10049886,
    8465274,
    6645519,
    4326986,
    2805623,
    1538659,
    703131,
    179003,
    20792
]
_female = [
    9621269,
    9798759,
    10311974,
    10408587,
    10936013,
    11690875,
    11349965,
    10756920,
    10176017,
    10084699,
    10258272,
    10840205,
    10619257,
    9353753,
    7709344,
    5400748,
    3655579,
    2372445,
    1348005,
    447633,
    76312
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)

In [ ]:
"""
_name = "Template"
_male = [
]
_female = [
]
_pyramid_df[_name] = np.array(_male) + np.array(_female)
"""
None

In [ ]:
df = _pyramid_df.drop(["Age_bin"], axis=1).sum(axis=0)
population_dict = df.to_dict()
pd.DataFrame(df, columns=["Total population"]).T

In [ ]:
df = _pyramid_df.copy()
series = df["Age_bin"].str.replace("+", "-122")
df[["Age_first", "Age_last"]] = series.str.split("-", expand=True).astype(np.int64)
df = df.drop("Age_bin", axis=1)
series = df["Age_last"]
df = df.apply(lambda x: x[:-2] / (x[-1] - x[-2] + 1), axis=1)
df["Age"] = series
df = pd.merge(df, pd.DataFrame({"Age": np.arange(0, 123, 1)}), on="Age", how="right", sort=True)
df = df.fillna(method="bfill").astype(np.int64)
df = df.set_index("Age")
pyramid_df = df.copy()
pyramid_df

## The number of days go out (template data)
**As a comment of this notebook, @marcoferrante estimated the number of days persons of each age group usually go out. Thank you for your kind cooperation!!**

In [ ]:
# @marcoferrante estimation
_period_of_life_list = [
    "nursery", "nursery school", "elementary school", "middle school",
    "high school", "university/work", "work", "work", "work", "work",
    "retired", "retired", "retired"
]
df = pd.DataFrame(
    {
        "Age_first": [0, 3, 6, 11, 14, 19, 26, 36, 46, 56, 66, 76, 86],
        "Age_last": [2, 5, 10, 13, 18, 25, 35, 45, 55, 65, 75, 85, 95],
        "Period_of_life": _period_of_life_list,
        "Days": [3, 5, 6, 6, 7, 7, 6, 5, 5, 5, 4, 3, 2]
    }
)
# Adjustment by author
df["Types"] = df["Period_of_life"].replace(
    {
        "nursery": "school",
        "nursery school": "school",
        "elementary school": "school",
        "middle school": "school",
        "high school": "school",
        "university/work": "school/work"
    }
)
df["School"] = df[["Types", "Days"]].apply(lambda x: x[1] if "school" in x[0] else 0, axis=1)
df["Office"] = df[["Types", "Days"]].apply(lambda x: x[1] if "work" in x[0] else 0, axis=1)
df["Others"] = df["Days"] - df[["School", "Office"]].sum(axis=1)
df.loc[df["Others"] < 0, "Others"] = 0
df.loc[df.index[1:5], "School"] -= 1
df.loc[df.index[1:5], "Others"] += 1
df.loc[df.index[5], ["School", "Office", "Others"]] = [3, 3, 1]
df[["School", "Office", "Others"]] = df[["Days", "School", "Office", "Others"]].apply(
    lambda x: x[1:] / sum(x[1:]) * x[0], axis=1
).astype(np.int64)
df.loc[df.index[6:10], "Others"] += 1
df = df.drop(["Days", "Types"], axis=1)
# Show dataset
_out_df = df.copy()
_out_df

For each country, population pyramid data will be combined to the table. The columns with countriy names are the portion of the total population.

In [ ]:
df = pyramid_df.cumsum()
countries = df.columns[:]
df = pd.merge(_out_df, df, left_on="Age_last", right_on="Age", how="left")
_first = df.loc[df.index[0], countries]
df[countries] = df[countries].diff()
df.loc[df.index[0], countries] = _first
df[countries] = df[countries].apply(lambda x: x / x.sum(), axis=0)
out_df = df.copy()
out_df

## Dataset

In [ ]:
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Functions
Here, we define the functions to use repeatedly in this notebook.

### Plotting

In [ ]:
def line_plot(df, title, ylabel="Cases", h=None, v=None,
              xlim=(None, None), ylim=(0, None), math_scale=True, y_logscale=False, y_integer=False):
    """
    Show chlonological change of the data.
    """
    ax = df.plot()
    if math_scale:
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style="sci",  axis="y",scilimits=(0, 0))
    if y_logscale:
        ax.set_yscale("log")
    if y_integer:
        fmt = matplotlib.ticker.ScalarFormatter(useOffset=False)
        fmt.set_scientific(False)
        ax.yaxis.set_major_formatter(fmt)
    ax.set_title(title)
    ax.set_xlabel(None)
    ax.set_ylabel(ylabel)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.legend(bbox_to_anchor=(1.02, 0), loc="lower left", borderaxespad=0)
    if h is not None:
        ax.axhline(y=h, color="black", linestyle="--")
    if v is not None:
        if not isinstance(v, list):
            v = [v]
        for value in v:
            ax.axvline(x=value, color="black", linestyle="--")
    plt.tight_layout()
    plt.show()

### Trend analysis

In [ ]:
def select_area(clean_df, places=None, excluded_places=None, group="Date"):
    """
    Select the records of the palces.
    @clean_df <pd.DataFrame>: the clean data
    @places <list[tuple(<str/None>, <str/None>)]: the list of places
        - if the list is None, all data will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    @excluded_places <list[tuple(<str/None>, <str/None>)]: the list of excluded places
        - if the list is None, all data in the "places" will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    @group <str or None>: group-by the group, or not perform (None)
    @return <pd.DataFrame>: index and columns are as same as @ncov_df
    """
    # Select the target records
    df = clean_df.copy()
    c_series = df["Country"]
    p_series = df["Province"]
    if places is not None:
        df = pd.DataFrame(columns=clean_df.columns)
        for (c, p) in places:
            if c is None:
                raise Exception("places: Country must be specified!")
            if p is None:
                new_df = clean_df.loc[c_series == c, :]
            else:
                new_df = clean_df.loc[(c_series == c) & (p_series == p), :]
            df = pd.concat([df, new_df], axis=0)
    if excluded_places is not None:
        for (c, p) in excluded_places:
            if c is None:
                raise Exception("excluded_places: Country must be specified!")
            if p is None:
                df = df.loc[c_series != c, :]
            else:
                c_df = df.loc[(c_series == c) & (p_series != p), :]
                other_df = df.loc[c_series != c, :]
                df = pd.concat([c_df, other_df], axis=0)
    if group is not None:
        df = df.groupby(group).sum().reset_index()
    return df

In [ ]:
def show_trend(ncov_df, variable="Confirmed", n_changepoints=2, places=None, excluded_places=None):
    """
    Show trend of log10(@variable) using fbprophet package.
    @ncov_df <pd.DataFrame>: the clean data
    @variable <str>: variable name to analyse
        - if Confirmed, use Infected + Recovered + Deaths
    @n_changepoints <int>: max number of change points
    @places <list[tuple(<str/None>, <str/None>)]: the list of places
        - if the list is None, all data will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    @excluded_places <list[tuple(<str/None>, <str/None>)]: the list of excluded places
        - if the list is None, all data in the "places" will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    """
    # Data arrangement
    df = select_area(ncov_df, places=places, excluded_places=excluded_places)
    if variable == "Confirmed":
        df["Confirmed"] = df[["Infected", "Recovered", "Deaths"]].sum(axis=1)
    df = df.loc[:, ["Date", variable]]
    df.columns = ["ds", "y"]
    # Log10(x)
    warnings.resetwarnings()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df["y"] = np.log10(df["y"]).replace([np.inf, -np.inf], 0)
    # fbprophet
    model = Prophet(growth="linear", daily_seasonality=False, n_changepoints=n_changepoints)
    model.fit(df)
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)
    # Create figure
    fig = model.plot(forecast)
    _ = add_changepoints_to_plot(fig.gca(), model, forecast)
    plt.title(f"log10({variable}) over time and chainge points")
    plt.ylabel(f"log10(the number of cases)")
    plt.xlabel("")

### Dataset arrangement

In [ ]:
def create_target_df(ncov_df, total_population, places=None,
                     excluded_places=None, start_date=None, date_format="%d%b%Y"):
    """
    Select the records of the palces, calculate the number of susceptible people,
     and calculate the elapsed time [day] from the start date of the target dataframe.
    @ncov_df <pd.DataFrame>: the clean data
    @total_population <int>: total population in the places
    @places <list[tuple(<str/None>, <str/None>)]: the list of places
        - if the list is None, all data will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    @excluded_places <list[tuple(<str/None>, <str/None>)]: the list of excluded places
        - if the list is None, all data in the "places" will be used
        - (str, str): both of country and province are specified
        - (str, None): only country is specified
        - (None, str) or (None, None): Error
    @start_date <str>: the start date or None
    @date_format <str>: format of @start_date
    @return <tuple(2 objects)>:
        - 1. start_date <pd.Timestamp>: the start date of the selected records
        - 2. target_df <pd.DataFrame>:
            - column T: elapsed time [min] from the start date of the dataset
            - column Susceptible: the number of patients who are in the palces but not infected/recovered/died
            - column Infected: the number of infected cases
            - column Recovered: the number of recovered cases
            - column Deaths: the number of death cases
    """
    # Select the target records
    df = select_area(ncov_df, places=places, excluded_places=excluded_places)
    if start_date is not None:
        df = df.loc[df["Date"] > datetime.strptime(start_date, date_format), :]
    start_date = df.loc[df.index[0], "Date"]
    # column T
    df["T"] = ((df["Date"] - start_date).dt.total_seconds() / 60).astype(int)
    # coluns except T
    df["Susceptible"] = total_population - df["Infected"] - df["Recovered"] - df["Deaths"]
    response_variables = ["Susceptible", "Infected", "Recovered", "Deaths"]
    # Return
    target_df = df.loc[:, ["T", *response_variables]]
    return (start_date, target_df)

### Numerical simulation
We will perform numerical analysis to solve the ODE using scipy.integrate.solve_ivp function.

In [ ]:
def simulation(model, initials, step_n, **params):
    """
    Solve ODE of the model.
    @model <ModelBase>: the model
    @initials <tuple[float]>: the initial values
    @step_n <int>: the number of steps
    """
    tstart, dt, tend = 0, 1, step_n
    sol = solve_ivp(
        fun=model(**params),
        # Implicit Runge-Kutta method of the Radau IIA family of order 5
        # method="Radau",
        t_span=[tstart, tend],
        y0=np.array(initials, dtype=np.float64),
        t_eval=np.arange(tstart, tend + dt, dt),
        dense_output=True
    )
    t_df = pd.Series(data=sol["t"], name="t")
    y_df = pd.DataFrame(data=sol["y"].T.copy(), columns=model.VARIABLES)
    sim_df = pd.concat([t_df, y_df], axis=1)
    return sim_df

### Parameter Estimation using Optuna

In [ ]:
class Estimator(object):
    def __init__(self, model, ncov_df, total_population, name=None, places=None,
                 excluded_places=None, start_date=None, date_format="%d%b%Y", **kwargs):
        """
        Set training data.
        @model <ModelBase>: the model
        @name <str>: name of the area
        @kwargs: fixed parameter of the model
        @the other params: See the function named create_target_df()
        """
        self.fixed_param_dict = kwargs.copy()
        dataset = model.create_dataset(
            ncov_df, total_population, places=places, excluded_places=excluded_places,
            start_date=start_date, date_format=date_format
        )
        self.start_time, self.initials, self.Tend, self.train_df = dataset
        self.total_population = total_population
        self.name = name
        self.model = model
        self.param_dict = dict()
        self.study = None
        self.optimize_df = None

    def run(self, n_trials=500):
        """
        Try estimation (optimization of parameters and tau).
        @n_trials <int>: the number of trials
        """
        if self.study is None:
            self.study = optuna.create_study(direction="minimize")
        self.study.optimize(
            lambda x: self.objective(x),
            n_trials=n_trials,
            n_jobs=-1
        )
        param_dict = self.study.best_params.copy()
        param_dict.update(self.fixed_param_dict)
        param_dict["R0"] = self.calc_r0()
        param_dict["score"] = self.score()
        param_dict.update(self.calc_days_dict())
        self.param_dict = param_dict.copy()
        return param_dict

    def history_df(self):
        """
        Return the hsitory of optimization.
        @return <pd.DataFrame>
        """
        optimize_df = self.study.trials_dataframe()
        optimize_df["time[s]"] = optimize_df["datetime_complete"] - optimize_df["datetime_start"]
        optimize_df["time[s]"] = optimize_df["time[s]"].dt.total_seconds()
        self.optimize_df = optimize_df.drop(["datetime_complete", "datetime_start"], axis=1)
        return self.optimize_df.sort_values("value", ascending=True)

    def history_graph(self):
        """
        Show the history of parameter search using pair-plot.
        """
        if self.optimize_df is None:
            self.history_df()
        df = self.optimize_df.copy()
        sns.pairplot(df.loc[:, df.columns.str.startswith("params_")], diag_kind="kde", markers="+")
        plt.show()

    def objective(self, trial):
        # Time
        if "tau" in self.fixed_param_dict.keys():
            tau = self.fixed_param_dict["tau"]
        else:
            tau = trial.suggest_int("tau", 1, 1440)
        # Apply adjusted Exponential Moving Average on the training data
        #.set_index("T").ewm(span=7, adjust=True).mean().reset_index()
        train_df_divided = self.train_df.copy()
        train_df_divided["t"] = (train_df_divided["T"] / tau).astype(np.int64) # int to np.int64
        # Parameters
        p_dict = dict()
        for (name, info) in self.model.param_dict(train_df_divided).items():
            if name in self.fixed_param_dict.keys():
                param = self.fixed_param_dict[name]
            elif info[0] == "float":
                param = trial.suggest_uniform(name, info[1], info[2])
            else:
                param = trial.suggest_int(name, info[1], info[2])
            p_dict[name] = param
        # Simulation
        t_end = train_df_divided.loc[train_df_divided.index[-1], "t"]
        sim_df = simulation(self.model, self.initials, step_n=t_end, **p_dict)
        return self.error_f(train_df_divided, sim_df)

    def error_f(self, train_df_divided, sim_df):
        """
        We need to minimize the difference of the observed values and estimated values.
        This function calculate the difference of the estimated value and obsereved value.
        """
        df = pd.merge(train_df_divided, sim_df, on="t", suffixes=("_observed", "_estimated"))
        diffs = [
            # Weighted Average: the recent data is more important
            p * np.average(
                abs(df[f"{v}_observed"] - df[f"{v}_estimated"]) / (df[f"{v}_observed"] * self.total_population + 1),
                weights=df["t"]
            )
            for (p, v) in zip(self.model.PRIORITIES, self.model.VARIABLES)
        ]
        return sum(diffs) * (self.total_population ** 2)

    def compare_df(self):
        """
        Show the taining data and simulated data in one dataframe.
        
        """
        est_dict = self.study.best_params.copy()
        est_dict.update(self.fixed_param_dict)
        tau = est_dict["tau"]
        est_dict.pop("tau")
        observed_df = self.train_df.drop("T", axis=1)
        observed_df["t"] = (self.train_df["T"] / tau).astype(int)
        t_end = observed_df.loc[observed_df.index[-1], "t"]
        sim_df = simulation(self.model, self.initials, step_n=t_end, **est_dict)
        df = pd.merge(observed_df, sim_df, on="t", suffixes=("_observed", "_estimated"))
        df = df.set_index("t")
        return df

    def compare_graph(self):
        """
        Compare obsereved and estimated values in graphs.
        """
        df = self.compare_df()
        use_variables = [
            v for (i, (p, v)) in enumerate(zip(self.model.PRIORITIES, self.model.VARIABLES))
            if p != 0 and i != 0
        ]
        val_len = len(use_variables) + 1
        fig, axes = plt.subplots(ncols=1, nrows=val_len, figsize=(9, 6 * val_len / 2))
        for (ax, v) in zip(axes.ravel()[1:],use_variables):
            df[[f"{v}_observed", f"{v}_estimated"]].plot.line(
                ax=ax, ylim=(0, None), sharex=True,
                title=f"{self.model.NAME}: Comparison of observed/estimated {v}(t)"
            )
            ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            ax.ticklabel_format(style="sci",  axis="y",scilimits=(0, 0))
            ax.legend(bbox_to_anchor=(1.02, 0), loc="lower left", borderaxespad=0)
        for v in use_variables:
            df[f"{v}_diff"] = df[f"{v}_observed"] - df[f"{v}_estimated"]
            df[f"{v}_diff"].plot.line(
                ax=axes.ravel()[0], sharex=True,
                title=f"{self.model.NAME}: observed - estimated"
            )
        axes.ravel()[0].axhline(y=0, color="black", linestyle="--")
        axes.ravel()[0].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        axes.ravel()[0].ticklabel_format(style="sci",  axis="y",scilimits=(0, 0))
        axes.ravel()[0].legend(bbox_to_anchor=(1.02, 0), loc="lower left", borderaxespad=0)
        fig.tight_layout()
        fig.show()
    
    def calc_r0(self):
        """
        Calculate R0.
        """
        est_dict = self.study.best_params.copy()
        est_dict.update(self.fixed_param_dict)
        est_dict.pop("tau")
        model_instance = self.model(**est_dict)
        return model_instance.calc_r0()

    def calc_days_dict(self):
        """
        Calculate 1/beta etc.
        """
        est_dict = self.study.best_params.copy()
        est_dict.update(self.fixed_param_dict)
        tau = est_dict["tau"]
        est_dict.pop("tau")
        model_instance = self.model(**est_dict)
        return model_instance.calc_days_dict(tau)

    def predict_df(self, step_n):
        """
        Predict the values in the future.
        @step_n <int>: the number of steps
        @return <pd.DataFrame>: predicted data for measurable variables.
        """
        est_dict = self.study.best_params.copy()
        est_dict.update(self.fixed_param_dict)
        tau = est_dict["tau"]
        est_dict.pop("tau")
        df = simulation(self.model, self.initials, step_n=step_n, **est_dict)
        df["Time"] = (df["t"] * tau).apply(lambda x: timedelta(minutes=x)) + self.start_time
        df = df.set_index("Time").drop("t", axis=1)
        df = (df * self.total_population).astype(np.int64)
        upper_cols = [n.upper() for n in df.columns]
        df.columns = upper_cols
        df = self.model.calc_variables_reverse(df).drop(upper_cols, axis=1)
        return df

    def predict_graph(self, step_n, name=None, excluded_cols=None):
        """
        Predict the values in the future and create a figure.
        @step_n <int>: the number of steps
        @name <str>: name of the area
        @excluded_cols <list[str]>: the excluded columns in the figure
        """
        if self.name is not None:
            name = self.name
        else:
            name = str() if name is None else name
        df = self.predict_df(step_n=step_n)
        if excluded_cols is not None:
            df = df.drop(excluded_cols, axis=1)
        r0 = self.param_dict["R0"]
        title = f"Prediction in {name} with {self.model.NAME} model: R0 = {r0}"
        line_plot(df, title, v= datetime.today(), h=self.total_population)

    def score(self):
        """
        Return the sum of differences of observed and estimated values devided by the number of steps.
        """
        variables = self.model.VARIABLES[:]
        compare_df = self.compare_df()
        score = 0
        for v in variables:
            score += abs(compare_df[f"{v}_observed"] - compare_df[f"{v}_estimated"]).sum()
        score = score / len(compare_df)
        return score

    def info(self):
        """
        Return Estimater information.
        @return <tupple[object]>:
            - <ModelBase>: model
            - <dict[str]=str>: name, total_population, start_time, tau
            - <dict[str]=float>: values of parameters of model
        """
        param_dict = self.study.best_params.copy()
        param_dict.update(self.fixed_param_dict)
        info_dict = {
            "name": self.name,
            "total_population": self.total_population,
            "start_time": self.start_time,
            "tau": param_dict["tau"],
            "initials": self.initials
        }
        param_dict.pop("tau")
        return (self.model, info_dict, param_dict)

### Description of math model

In [ ]:
class ModelBase(object):
    NAME = "Model"
    VARIABLES = ["x"]
    PRIORITIES = np.array([1])
    QUANTILE_RANGE = [0.3, 0.7]

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        """
        Define parameters without tau. This function should be overwritten.
        @train_df_divided <pd.DataFrame>:
            - column: t and non-dimensional variables
        @q_range <list[float, float]>: quantile rage of the parameters calculated by the data
        @return <dict[name]=(type, min, max):
            @type <str>: "float" or "int"
            @min <float/int>: min value
            @max <float/int>: max value
        """
        param_dict = dict()
        return param_dict

    @staticmethod
    def calc_variables(df):
        """
        Calculate the variables of the model.
        This function should be overwritten.
        @df <pd.DataFrame>
        @return <pd.DataFrame>
        """
        return df

    @staticmethod
    def calc_variables_reverse(df):
        """
        Calculate measurable variables using the variables of the model.
        This function should be overwritten.
        @df <pd.DataFrame>
        @return <pd.DataFrame>
        """
        return df

    @classmethod
    def create_dataset(cls, ncov_df, total_population, places=None,
                       excluded_places=None, start_date=None, date_format="%d%b%Y"):
        """
        Create dataset with the model-specific varibles.
        The variables will be divided by total population.
        The column names (not include T) will be lower letters.
        @params: See the function named create_target_df()
        @return <tuple(objects)>:
            - start_date <pd.Timestamp>
            - initials <tuple(float)>: the initial values
            - Tend <int>: the last value of T
            - df <pd.DataFrame>: the dataset
        """
        start_date, target_df = create_target_df(
            ncov_df, total_population, places=places, excluded_places=excluded_places,
            start_date=start_date, date_format=date_format
        )
        df = cls.calc_variables(target_df).set_index("T") / total_population
        df.columns = [n.lower() for n in df.columns]
        initials = df.iloc[0, :].values
        df = df.reset_index()
        Tend = df.iloc[-1, 0]
        return (start_date, initials, Tend, df)

    def calc_r0(self):
        """
        Calculate R0. This function should be overwritten.
        """
        return None

    def calc_days_dict(self, tau):
        """
        Calculate 1/beta [day] etc.
        This function should be overwritten.
        @param tau <int>: tau value [hour]
        """
        return dict()

#### SIR model

In [ ]:
class SIR(ModelBase):
    NAME = "SIR"
    VARIABLES = ["x", "y", "z"]
    PRIORITIES = np.array([1, 1, 1])

    def __init__(self, rho, sigma):
        super().__init__()
        self.rho = float(rho)
        self.sigma = float(sigma)

    def __call__(self, t, X):
        # x, y, z = [X[i] for i in range(len(self.VARIABLES))]
        # dxdt = - self.rho * x * y
        # dydt = self.rho * x * y - self.sigma * y
        # dzdt = self.sigma * y
        dxdt = - self.rho * X[0] * X[1]
        dydt = self.rho * X[0] * X[1] - self.sigma * X[1]
        dzdt = self.sigma * X[1]
        return np.array([dxdt, dydt, dzdt])

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        param_dict = super().param_dict()
        q_range = super().QUANTILE_RANGE[:] if q_range is None else q_range
        if train_df_divided is None:
            param_dict["rho"] = ("float", 0, 1)
            param_dict["sigma"] = ("float", 0, 1)
        else:
            df = train_df_divided.copy()
            # rho = - (dx/dt) / x / y
            rho_series = 0 - df["x"].diff() / df["t"].diff() / df["x"] / df["y"]
            param_dict["rho"] = ("float", *rho_series.quantile(q_range))
            # sigma = (dz/dt) / y
            sigma_series = df["z"].diff() / df["t"].diff() / df["y"]
            param_dict["sigma"] = ("float", *sigma_series.quantile(q_range))
        return param_dict

    @staticmethod
    def calc_variables(df):
        df["X"] = df["Susceptible"]
        df["Y"] = df["Infected"]
        df["Z"] = df["Recovered"] + df["Deaths"]
        return df.loc[:, ["T", "X", "Y", "Z"]]

    @staticmethod
    def calc_variables_reverse(df):
        df["Susceptible"] = df["X"]
        df["Infected"] = df["Y"]
        df["Recovered/Deaths"] = df["Z"]
        return df

    def calc_r0(self):
        if self.sigma == 0:
            return np.nan
        r0 = self.rho / self.sigma
        return round(r0, 2)

    def calc_days_dict(self, tau):
        _dict = dict()
        _dict["1/beta [day]"] = int(tau / 24 / 60 / self.rho)
        _dict["1/gamma [day]"] = int(tau / 24 / 60 / self.sigma)
        return _dict

#### SIR-D model

In [ ]:
class SIRD(ModelBase):
    NAME = "SIR-D"
    VARIABLES = ["x", "y", "z", "w"]
    PRIORITIES = np.array([1, 10, 10, 2])

    def __init__(self, kappa, rho, sigma):
        super().__init__()
        self.kappa = float(kappa)
        self.rho = float(rho)
        self.sigma = float(sigma)

    def __call__(self, t, X):
        # x, y, z, w = [X[i] for i in range(len(self.VARIABLES))]
        # dxdt = - self.rho * x * y
        # dydt = self.rho * x * y - (self.sigma + self.kappa) * y
        # dzdt = self.sigma * y
        # dwdt = self.kappa * y
        dxdt = - self.rho * X[0] * X[1]
        dydt = self.rho * X[0] * X[1] - (self.sigma + self.kappa) * X[1]
        dzdt = self.sigma * X[1]
        dwdt = self.kappa * X[1]
        return np.array([dxdt, dydt, dzdt, dwdt])

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        param_dict = super().param_dict()
        q_range = super().QUANTILE_RANGE[:] if q_range is None else q_range
        if train_df_divided is None:
            param_dict["kappa"] = ("float", 0, 1)
            param_dict["rho"] = ("float", 0, 1)
            param_dict["sigma"] = ("float", 0, 1)
        else:
            df = train_df_divided.copy()
            # kappa = (dw/dt) / y
            kappa_series = df["w"].diff() / df["t"].diff() / df["y"]
            param_dict["kappa"] = ("float", *kappa_series.quantile(q_range))
            # rho = - (dx/dt) / x / y
            rho_series = 0 - df["x"].diff() / df["t"].diff() / df["x"] / df["y"]
            param_dict["rho"] = ("float", *rho_series.quantile(q_range))
            # sigma = (dz/dt) / y
            sigma_series = df["z"].diff() / df["t"].diff() / df["y"]
            param_dict["sigma"] = ("float", *sigma_series.quantile(q_range))
        return param_dict

    @staticmethod
    def calc_variables(df):
        df["X"] = df["Susceptible"]
        df["Y"] = df["Infected"]
        df["Z"] = df["Recovered"]
        df["W"] = df["Deaths"]
        return df.loc[:, ["T", "X", "Y", "Z", "W"]]

    @staticmethod
    def calc_variables_reverse(df):
        df["Susceptible"] = df["X"]
        df["Infected"] = df["Y"]
        df["Recovered"] = df["Z"]
        df["Deaths"] = df["W"]
        return df

    def calc_r0(self):
        try:
            r0 = self.rho / (self.sigma + self.kappa)
        except ZeroDivisionError:
            return np.nan
        return round(r0, 2)


    def calc_days_dict(self, tau):
        _dict = dict()
        if self.kappa == 0:
            _dict["1/alpha2 [day]"] = 0
        else:
            _dict["1/alpha2 [day]"] = int(tau / 24 / 60 / self.kappa)
        _dict["1/beta [day]"] = int(tau / 24 / 60 / self.rho)
        if self.sigma == 0:
            _dict["1/gamma [day]"] = 0
        else:
            _dict["1/gamma [day]"] = int(tau / 24 / 60 / self.sigma)
        return _dict

#### SIR-F model

In [ ]:
class SIRF(ModelBase):
    NAME = "SIR-F"
    VARIABLES = ["x", "y", "z", "w"]
    PRIORITIES = np.array([1, 10, 10, 2])

    def __init__(self, theta, kappa, rho, sigma):
        super().__init__()
        self.theta = float(theta)
        self.kappa = float(kappa)
        self.rho = float(rho)
        self.sigma = float(sigma)

    def __call__(self, t, X):
        # x, y, z, w = [X[i] for i in range(len(self.VARIABLES))]
        # dxdt = - self.rho * x * y
        # dydt = self.rho * (1 - self.theta) * x * y - (self.sigma + self.kappa) * y
        # dzdt = self.sigma * y
        # dwdt = self.rho * self.theta * x * y + self.kappa * y
        dxdt = - self.rho * X[0] * X[1]
        dydt = self.rho * (1 - self.theta) * X[0] * X[1] - (self.sigma + self.kappa) * X[1]
        dzdt = self.sigma * X[1]
        dwdt = self.rho * self.theta * X[0] * X[1] + self.kappa * X[1]
        return np.array([dxdt, dydt, dzdt, dwdt])

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        param_dict = super().param_dict()
        q_range = super().QUANTILE_RANGE[:] if q_range is None else q_range
        param_dict["theta"] = ("float", 0, 1)
        param_dict["kappa"] = ("float", 0, 1)
        if train_df_divided is None:
            param_dict["rho"] = ("float", 0, 1)
            param_dict["sigma"] = ("float", 0, 1)
        else:
            df = train_df_divided.copy()
            # rho = - (dx/dt) / x / y
            rho_series = 0 - df["x"].diff() / df["t"].diff() / df["x"] / df["y"]
            param_dict["rho"] = ("float", *rho_series.quantile(q_range))
            # sigma = (dz/dt) / y
            sigma_series = df["z"].diff() / df["t"].diff() / df["y"]
            param_dict["sigma"] = ("float", *sigma_series.quantile(q_range))
        return param_dict

    @staticmethod
    def calc_variables(df):
        df["X"] = df["Susceptible"]
        df["Y"] = df["Infected"]
        df["Z"] = df["Recovered"]
        df["W"] = df["Deaths"]
        return df.loc[:, ["T", "X", "Y", "Z", "W"]]

    @staticmethod
    def calc_variables_reverse(df):
        df["Susceptible"] = df["X"]
        df["Infected"] = df["Y"]
        df["Recovered"] = df["Z"]
        df["Fatal"] = df["W"]
        return df

    def calc_r0(self):
        try:
            r0 = self.rho * (1 - self.theta) / (self.sigma + self.kappa)
        except ZeroDivisionError:
            return np.nan
        return round(r0, 2)

    def calc_days_dict(self, tau):
        _dict = dict()
        _dict["alpha1 [-]"] = round(self.theta, 3)
        if self.kappa == 0:
            _dict["1/alpha2 [day]"] = 0
        else:
            _dict["1/alpha2 [day]"] = int(tau / 24 / 60 / self.kappa)
        _dict["1/beta [day]"] = int(tau / 24 / 60 / self.rho)
        if self.sigma == 0:
            _dict["1/gamma [day]"] = 0
        else:
            _dict["1/gamma [day]"] = int(tau / 24 / 60 / self.sigma)
        return _dict

#### SEWIR-F model

In [ ]:
class SEWIRF(ModelBase):
    NAME = "SEWIR-F"
    VARIABLES = ["x1", "x2", "x3", "y", "z", "w"]
    PRIORITIES = np.array([0, 0, 0, 10, 10, 2])

    def __init__(self, theta, kappa, rho1, rho2, rho3, sigma):
        super().__init__()
        self.theta = float(theta)
        self.kappa = float(kappa)
        self.rho1 = float(rho1)
        self.rho2 = float(rho2)
        self.rho3 = float(rho3)
        self.sigma = float(sigma)

    def __call__(self, t, X):
        # x1, x2, y, z, w = [X[i] for i in range(len(self.VARIABLES))]
        # dx1dt = - self.rho1 * x1 * (x3 + y)
        # dx2dt = self.rho1 * x1 * (x3 + y) - self.rho2 * x2
        # dx3dt = self.rho2 * x2 - self.rho3 * x3
        # dydt = self.rho3 * (1 - self.theta) * x3 - (self.sigma + self.kappa) * y
        # dzdt = self.sigma * y
        # dwdt = self.rho3 * self.theta * x3 + self.kappa * y
        dx1dt = - self.rho1 * X[0] * (X[2] + X[3])
        dx2dt = self.rho1 * X[0] * (X[2] + X[3]) - self.rho2 * X[1]
        dx3dt = self.rho2 * X[1] - self.rho3 * X[2]
        dydt = self.rho3 * (1 - self.theta) * X[2] - (self.sigma + self.kappa) * X[3]
        dzdt = self.sigma * X[3]
        dwdt = self.rho3 * self.theta * X[2] + self.kappa * X[3]
        return np.array([dx1dt, dx2dt, dx3dt, dydt, dzdt, dwdt])

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        param_dict = super().param_dict()
        q_range = super().QUANTILE_RANGE[:] if q_range is None else q_range
        param_dict["theta"] = ("float", 0, 1)
        param_dict["kappa"] = ("float", 0, 1)
        param_dict["rho1"] = ("float", 0, 1)
        param_dict["rho2"] = ("float", 0, 1)
        param_dict["rho3"] = ("float", 0, 1)
        if train_df_divided is None:
            param_dict["sigma"] = ("float", 0, 1)
        else:
            df = train_df_divided.copy()
            # sigma = (dz/dt) / y
            sigma_series = df["z"].diff() / df["t"].diff() / df["y"]
            param_dict["sigma"] = ("float", *sigma_series.quantile(q_range))
        return param_dict

    @staticmethod
    def calc_variables(df):
        df["X1"] = df["Susceptible"]
        df["X2"] = 0
        df["X3"] = 0
        df["Y"] = df["Infected"]
        df["Z"] = df["Recovered"]
        df["W"] = df["Deaths"]
        return df.loc[:, ["T", "X1", "X2", "X3", "Y", "Z", "W"]]

    @staticmethod
    def calc_variables_reverse(df):
        df["Susceptible"] = df["X1"]
        df["Infected"] = df["Y"]
        df["Recovered"] = df["Z"]
        df["Fatal"] = df["W"]
        df["Exposed"] = df["X2"]
        df["Waiting"] = df["X3"]
        return df

    def calc_r0(self):
        try:
            r0 = self.rho1 * (1 - self.theta) / (self.sigma + self.kappa)
        except ZeroDivisionError:
            return np.nan
        return round(r0, 2)

    def calc_days_dict(self, tau):
        _dict = dict()
        _dict["alpha1 [-]"] = round(self.theta, 3)
        if self.kappa == 0:
            _dict["1/alpha2 [day]"] = 0
        else:
            _dict["1/alpha2 [day]"] = int(tau / 24 / 60 / self.kappa)
        _dict["1/beta1 [day]"] = int(tau / 24 / 60 / self.rho1)
        _dict["1/beta2 [day]"] = int(tau / 24 / 60 / self.rho2)
        _dict["1/beta3 [day]"] = int(tau / 24 / 60 / self.rho3)
        if self.sigma == 0:
            _dict["1/gamma [day]"] = 0
        else:
            _dict["1/gamma [day]"] = int(tau / 24 / 60 / self.sigma)
        return _dict

#### SIR-FV model

In [ ]:
class SIRFV(ModelBase):
    NAME = "SIR-FV"
    VARIABLES = ["x", "y", "z", "w"]
    PRIORITIES = np.array([1, 10, 10, 2])

    def __init__(self, theta, kappa, rho, sigma, omega=None, n=None, v_per_day=None):
        """
        (n and v_per_day) or omega must be applied.
        @n <float or int>: total population
        @v_par_day <float or int>: vacctinated persons per day
        """
        super().__init__()
        self.theta = float(theta)
        self.kappa = float(kappa)
        self.rho = float(rho)
        self.sigma = float(sigma)
        if omega is None:
            try:
                self.omega = float(v_per_day) / float(n)
            except TypeError:
                s = "Neither (n and va_per_day) nor omega must be applied!"
                raise TypeError(s)
        else:
            self.omega = float(omega)

    def __call__(self, t, X):
        # x, y, z, w = [X[i] for i in range(len(self.VARIABLES))]
        # x with vacctination
        dxdt = - self.rho * X[0] * X[1] - self.omega
        dxdt = 0 - X[0] if X[0] + dxdt < 0 else dxdt
        # y, z, w
        dydt = self.rho * (1 - self.theta) * X[0] * X[1] - (self.sigma + self.kappa) * X[1]
        dzdt = self.sigma * X[1]
        dwdt = self.rho * self.theta * X[0] * X[1] + self.kappa * X[1]
        return np.array([dxdt, dydt, dzdt, dwdt])

    @classmethod
    def param_dict(cls, train_df_divided=None, q_range=None):
        param_dict = super().param_dict()
        q_range = super().QUANTILE_RANGE[:] if q_range is None else q_range
        param_dict["theta"] = ("float", 0, 1)
        param_dict["kappa"] = ("float", 0, 1)
        param_dict["omega"] = ("float", 0, 1)
        if train_df_divided is None:
            param_dict["rho"] = ("float", 0, 1)
            param_dict["sigma"] = ("float", 0, 1)
        else:
            df = train_df_divided.copy()
            # rho = - (dx/dt) / x / y
            rho_series = 0 - df["x"].diff() / df["t"].diff() / df["x"] / df["y"]
            param_dict["rho"] = ("float", *rho_series.quantile(q_range))
            # sigma = (dz/dt) / y
            sigma_series = df["z"].diff() / df["t"].diff() / df["y"]
            param_dict["sigma"] = ("float", *sigma_series.quantile(q_range))
        return param_dict

    @staticmethod
    def calc_variables(df):
        df["X"] = df["Susceptible"]
        df["Y"] = df["Infected"]
        df["Z"] = df["Recovered"]
        df["W"] = df["Deaths"]
        return df.loc[:, ["T", "X", "Y", "Z", "W"]]

    @staticmethod
    def calc_variables_reverse(df):
        df["Susceptible"] = df["X"]
        df["Infected"] = df["Y"]
        df["Recovered"] = df["Z"]
        df["Fatal"] = df["W"]
        df["Immuned"] = 1 - df[["X", "Y", "Z", "W"]].sum(axis=1)
        return df

    def calc_r0(self):
        try:
            r0 = self.rho * (1 - self.theta) / (self.sigma + self.kappa)
        except ZeroDivisionError:
            return np.nan
        return round(r0, 2)

    def calc_days_dict(self, tau):
        _dict = dict()
        _dict["alpha1 [-]"] = round(self.theta, 3)
        if self.kappa == 0:
            _dict["1/alpha2 [day]"] = 0
        else:
            _dict["1/alpha2 [day]"] = int(tau / 24 / 60 / self.kappa)
        _dict["1/beta [day]"] = int(tau / 24 / 60 / self.rho)
        if self.sigma == 0:
            _dict["1/gamma [day]"] = 0
        else:
            _dict["1/gamma [day]"] = int(tau / 24 / 60 / self.sigma)
        return _dict

### Prediction of the data using some models

In [ ]:
class Predicter(object):
    """
    Predict the future using models.
    """
    def __init__(self, name, total_population, start_time, tau, initials, date_format="%d%b%Y"):
        """
        @name <str>: place name
        @total_population <int>: total population
        @start_time <datatime>: the start time
        @tau <int>: tau value (time step)
        @initials <list/tupple/np.array[float]>: initial values of the first model
        @date_format <str>: date format to display in figures
        """
        self.name = name
        self.total_population = total_population
        self.start_time = start_time
        self.tau = tau
        self.date_format = date_format
        # Un-fixed
        self.last_time = start_time
        self.axvlines = list()
        self.initials = initials
        self.df = pd.DataFrame()
        self.title_list = list()
        self.reverse_f = lambda x: x

    def add(self, model, end_day_n=None, count_from_last=False, vline=True, **param_dict):
        """
        @model <ModelBase>: the epidemic model
        @end_day_n <int/None>: day number of the end date (0, 1, 2,...), or None (now)
            - if @count_from_last <bool> is True, start point will be the last date registered to Predicter
        @vline <bool>: if True, vertical line will be shown at the end date
        @**param_dict <dict>: keyword arguments of the model
        """
        # Validate day nubber, and calculate step number
        if end_day_n is None:
            end_time = datetime.now()
        else:
            if count_from_last:
                end_time = self.last_time + timedelta(days=end_day_n)
            else:
                end_time = self.start_time + timedelta(days=end_day_n)
        if end_time <= self.last_time:
            raise Exception(f"Model on {end_time.strftime(self.date_format)} has been registered!")
        step_n = int((end_time - self.last_time).total_seconds() / 60 / self.tau)
        self.last_time = end_time
        # Perform simulation
        new_df = simulation(model, self.initials, step_n=step_n, **param_dict)
        new_df["t"] = new_df["t"] + len(self.df)
        self.df = pd.concat([self.df, new_df], axis=0).fillna(0)
        self.initials = new_df.set_index("t").iloc[-1, :]
        # For title
        if vline:
            self.axvlines.append(end_time)
            r0 = model(**param_dict).calc_r0()
            self.title_list.append(
                f"{model.NAME}({r0}, -{end_time.strftime(self.date_format)})"
            )
        # Update reverse function (X, Y,.. to Susceptible, Infected,...)
        self.reverse_f = model.calc_variables_reverse
        return self

    def restore_df(self):
        """
        Return the dimentional simulated data.
        @return <pd.DataFrame>
        """
        df = self.df.copy()
        df["Time"] = self.start_time + df["t"].apply(lambda x: timedelta(minutes=x * self.tau))
        df = df.drop("t", axis=1).set_index("Time") * self.total_population
        df = df.astype(np.int64)
        upper_cols = [n.upper() for n in df.columns]
        df.columns = upper_cols
        df = self.reverse_f(df).drop(upper_cols, axis=1)
        return df

    def restore_graph(self, drop_cols=None, **kwargs):
        """
        Show the dimentional simulate data as a figure.
        @drop_cols <list[str]>: the columns not to be shown
        @kwargs: keyword arguments of line_plot() function
        """
        df = self.restore_df()
        if drop_cols is not None:
            df = df.drop(drop_cols, axis=1)
        axvlines = [datetime.now(), *self.axvlines] if len(self.axvlines) == 1 else self.axvlines[:]
        line_plot(
            df,
            title=f"{self.name}: {', '.join(self.title_list)}",
            v=axvlines[:-1],
            h=self.total_population,
            **kwargs
        )

## Raw data

In [ ]:
raw = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv")
raw.tail()

In [ ]:
raw.info()

In [ ]:
raw.describe()

In [ ]:
pd.DataFrame(raw.isnull().sum()).T

In [ ]:
", ".join(raw["Country/Region"].unique().tolist())

In [ ]:
pprint(raw.loc[raw["Country/Region"] == "Others", "Province/State"].unique().tolist(), compact=True)

## Data Cleening: the number of cases
Note: "Infected" = "Confirmed" - "Deaths" - "Recovered"

In [ ]:
data_cols = ["Infected", "Deaths", "Recovered"]
rate_cols = ["Fatal per Confirmed", "Recovered per Confirmed", "Fatal per (Fatal or Recovered)"]
variable_dict = {"Susceptible": "S", "Infected": "I", "Recovered": "R", "Deaths": "D"}

In [ ]:
ncov_df = raw.rename({"ObservationDate": "Date", "Province/State": "Province"}, axis=1)
ncov_df["Date"] = pd.to_datetime(ncov_df["Date"])
ncov_df["Country"] = ncov_df["Country/Region"].replace(
    {
        "Mainland China": "China",
        "Hong Kong SAR": "Hong Kong",
        "Taipei and environs": "Taiwan",
        "Iran (Islamic Republic of)": "Iran",
        "Republic of Korea": "South Korea",
        "Republic of Ireland": "Ireland",
        "Macao SAR": "Macau",
        "Russian Federation": "Russia",
        "Republic of Moldova": "Moldova",
        "Taiwan*": "Taiwan",
        "Cruise Ship": "Others",
        "United Kingdom": "UK",
        "Viet Nam": "Vietnam",
        "Czechia": "Czech Republic",
        "St. Martin": "Saint Martin",
        "Cote d'Ivoire": "Ivory Coast",
        "('St. Martin',)": "Saint Martin",
        "Congo (Kinshasa)": "Congo",
    }
)
ncov_df["Province"] = ncov_df["Province"].fillna("-").replace(
    {
        "Cruise Ship": "Diamond Princess cruise ship",
        "Diamond Princess": "Diamond Princess cruise ship"
    }
)
ncov_df["Infected"] = ncov_df["Confirmed"] - ncov_df["Deaths"] - ncov_df["Recovered"]
ncov_df[data_cols] = ncov_df[data_cols].astype(np.int64)
ncov_df = ncov_df.loc[:, ["Date", "Country", "Province", *data_cols]]
ncov_df.tail()

In [ ]:
ncov_df.info()

In [ ]:
ncov_df.describe(include="all").fillna("-")

In [ ]:
pd.DataFrame(ncov_df.isnull().sum()).T

In [ ]:
", ".join(ncov_df["Country"].unique().tolist())

## Visualize total data except China

In [ ]:
total_df = ncov_df.loc[ncov_df["Country"] != "China", :].groupby("Date").sum()
total_df[rate_cols[0]] = total_df["Deaths"] / total_df[data_cols].sum(axis=1)
total_df[rate_cols[1]] = total_df["Recovered"] / total_df[data_cols].sum(axis=1)
total_df[rate_cols[2]] = total_df["Deaths"] / (total_df["Deaths"] + total_df["Recovered"])
total_df.tail()

In [ ]:
f"{(total_df.index.max() - total_df.index.min()).days} days have passed from the date of the first record."

In [ ]:
line_plot(total_df[data_cols], "Cases over time (Total except China)")

In [ ]:
line_plot(total_df[rate_cols], "Rate over time (Total except China)", ylabel="", math_scale=False)

In [ ]:
total_df[rate_cols].plot.kde()
plt.title("Kernel density estimation of the rates (Total except China)")
plt.show()

In [ ]:
total_df[rate_cols].describe().T

## Example of dataset to create math model
 * T means elapsed time [min] from the start date.
 * Susceptible means the patients who are in the area but not infected/recovered/died.

In [ ]:
train_start_date, train_df = create_target_df(
    ncov_df, population_dict["Global"] - population_dict["China"], excluded_places=[("China", None)]
)
train_start_date.strftime(time_format)

In [ ]:
train_df.tail()

## Correlation of variables

In [ ]:
df = train_df.rename(variable_dict, axis=1)
for (_, v) in variable_dict.items():
    df[f"d{v}/dT"] = df[v].diff() / df["T"].diff()
    if v == "S":
        N = population_dict["Global"] - population_dict["China"]
        df["-(dS/dT)*(N/S)"] = 0 - df["S"].diff() / df["T"].diff() / df["S"] * N
df.set_index("T").corr().loc[variable_dict.values(), :].style.background_gradient(axis=None)

* Variables ($I, R, D$) shows high correlation with each other.
* $-\mathrm{d}S/\mathrm{d}T * (N/S)=\frac{S_{T+\Delta T} - S_T}{\Delta T} \times \frac{N}{S}$, $\mathrm{d}I/\mathrm{d}T=\frac{I_{T+\Delta T} - I_T}{\Delta T}$, $\mathrm{d}R/\mathrm{d}T=\frac{R_{T+\Delta T} - R_T}{\Delta T}$ and $\mathrm{d}D/\mathrm{d}T=\frac{D_{T+\Delta T} - D_T}{\Delta T}$ show high correlation with I, where N is the total population.

In [ ]:
df[["I", "-(dS/dT)*(N/S)", "dI/dT", "dR/dT", "dD/dT"]].tail()

In [ ]:
sns.lmplot(
    x="I", y="value", col="diff", sharex=False, sharey=False,
    data=df[["I", "-(dS/dT)*(N/S)", "dI/dT", "dR/dT", "dD/dT"]].melt(id_vars="I", var_name="diff")
)
plt.show()

## Data cleaning: Linelist (COVID19_open_line_list.csv)

In [ ]:
linelist_open_raw = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv")
linelist_open_raw.info()

In [ ]:
df = linelist_open_raw.loc[:, ~linelist_open_raw.columns.str.startswith("Unnamed:")]
df = df.dropna(axis=0, how="all")
df = df.drop(
    [
        # Unnecessary in this notebook
        "ID", "wuhan(0)_not_wuhan(1)", "admin3", "admin2", "admin1", "country_new", "admin_id",
        "data_moderator_initials", "source", "location", "lives_in_Wuhan", "notes_for_discussion",
        "sequence_available", "reported_market_exposure",
        # Maybe useful, but un-used
        "city", "latitude", "longitude", "geo_resolution", "additional_information",
        "travel_history_dates", "travel_history_location", 
    ],
    axis=1
)
# Personal
age = linelist_open_raw["age"].str.split("-", expand=True)
age[0] = pd.to_numeric(age[0], errors="coerce")
age[1] = pd.to_numeric(age[1], errors="coerce")
df["Age"] = age.mean(axis=1)
df["Age"] = df["Age"].fillna(df["Age"].median()).astype(np.int64)
df["Sex"] = df["sex"].fillna("-").str.replace("4000", "-").str.capitalize()
# Place
df["Country"] = df["country"].fillna("-")
df["Province"] = df["province"].fillna("-")
# Onset Date
series = df["date_onset_symptoms"].str.replace("end of December 2019", "31.12.2019").replace("-25.02.2020", "25.02.2020")
series = series.replace("20.02.220", "20.02.2020").replace("none", np.NaN).replace("10.01.2020 - 22.01.2020", np.NaN)
df["Onset_date"] = pd.to_datetime(series)
# Hospitalized date
series = df["date_admission_hospital"].replace("18.01.2020 - 23.01.2020", np.NaN)
df["Hospitalized_date"] = pd.to_datetime(series)
# Confirmed date
series = df["date_confirmation"].replace("25.02.2020-26.02.2020", np.NaN)
df["Confirmed_date"] = pd.to_datetime(series)
# Symptoms/events
df["Symptoms"] = df["symptoms"].fillna("-").str.lower()
# Underlying disease
df["Underlying_disease"] = df[["chronic_disease_binary", "chronic_disease"]].apply(
    lambda x: "No" if x[0] == 0 else x[1] if x[1] is not np.NaN else "-",
    axis=1
).str.strip(";").str.replace("; ", ",").str.replace(", ", ",")
# Outcome
df["Outcome"] = df["outcome"].replace(
    {
        "discharge": "discharged", "Discharged": "discharged", "death": "died",
        "critical condition, intubated as of 14.02.2020": "severe",
        "treated in an intensive care unit (14.02.2020)": "severe", "05.02.2020": "-",
        "Symptoms only improved with cough. Currently hospitalized for follow-up.": "stable"
    }
).fillna("-")
series = df["date_death_or_discharge"].replace("discharge", np.NaN)
df["Closed_date"] = pd.to_datetime(series)
# Show
use_cols = [
    "Age", "Sex", "Country", "Province", "Onset_date", "Hospitalized_date", "Confirmed_date", 
    "Symptoms", "Underlying_disease", "Outcome", "Closed_date"
]
open_linelist_df = df.loc[:, use_cols]
open_linelist_df.head()

## Data cleaning: Linelist (COVID19_line_list_data.csv)
Linelist in clinical trials is a list of many case reports.

In [ ]:
linelist_raw = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv")
linelist_raw.info()

In [ ]:
linelist_raw.head()

In [ ]:
df = linelist_raw.loc[:, ~linelist_raw.columns.str.startswith("Unnamed:")]
df = df.drop(["id", "case_in_country", "summary", "source", "link"], axis=1)
# Date
case_date_dict = {
    "reporting date": "Confirmed_date",
    "exposure_start": "Exposed_date",
    "exposure_end": "Quarantined_date",
    "hosp_visit_date": "Hospitalized_date",
    "symptom_onset": "Onset_date",
    "death": "Deaths_date",
    "recovered": "Recovered_date"    
}
df["death"] = df["death"].replace({"0": "", "1": ""})
df["recovered"] = df["recovered"].replace({"0": "", "1": "", "12/30/1899": "12/30/2019"})
for (col, _) in case_date_dict.items():
    df[col] = pd.to_datetime(df[col])
df = df.rename(case_date_dict, axis=1)
# Location
df["Country"] = df["country"].fillna("-")
df["Province"] = df["location"].fillna("-")
df["Province"] = df[["Country", "Province"]].apply(lambda x: "-" if x[0] == x[1] else x[1], axis=1)
# Personal
df["Gender"] = df["gender"].fillna("-").str.capitalize()
df["Age"] = df["age"].fillna(df["age"].median()).astype(np.int64) ## Fill in NA with median
df["From_Wuhan"] = df["from Wuhan"]
df["To_Wuhan"] = df["visiting Wuhan"]
# Medical
df["Events"] = df["symptom"].fillna("-")
# Order of columns
linelist_df = df.loc[
    :,
    [
        "Country", "Province",
        "Exposed_date", "Onset_date", "Hospitalized_date", "Confirmed_date", "Quarantined_date", "Deaths_date", "Recovered_date",
        "Events",
        "Gender", "Age", "From_Wuhan", "To_Wuhan"
    ]
]
linelist_df.tail()

In [ ]:
linelist_df.info()

In [ ]:
linelist_df.describe(include="all").fillna("-")

# Trend analysis
Using fbprophet package, we will find changing points of log10(comfirmed/deaths/recovered).

## Trend of log10(Confirmed)

In [ ]:
show_trend(ncov_df, variable="Confirmed", excluded_places=[("China", None)])

**No slope change points were found for the number of confirmed cases.**
<!--**The incline of the number of confirmed cases was slightly grow large on 01Mar2020. I will keep a watchful eye on the trend.**-->

## Trend of log10(Deaths)

In [ ]:
show_trend(ncov_df, variable="Deaths", excluded_places=[("China", None)])

**Slope of the number of fatal cases was changed on 04Feb2020. This change was caused by the first report of fatal case.**

## Trend of log10(Recovered)

In [ ]:
show_trend(ncov_df, variable="Recovered", excluded_places=[("China", None)])

**No slope change points were found for the number of recovered cases.**

# Improvement of math model
In this section, we will create a mathematical model derived from SIR model. The dataset of total except China will be used here as an example.

## Prediction with SIR model
To understand the trend of infection, we will use mathematical epidemic model. Let's start discussion using a basic model named SIR.

### What is SIR model?
SIR model is a simple mathematical model to understand outbreak of infectious diseases.  
[The SIR epidemic model - Learning Scientific Programming with Python](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/)

 * S: Susceptible (=All - Confirmed)
 * I: Infected (=Confirmed - Recovered - Deaths)
 * R: Recovered or fatal (=Recovered + Deaths)
 
Note: THIS IS NOT THE GENERAL MODEL!  
Though R in SIR model is "Recovered and have immunity", I defined "R as Recovered or fatal". This is because mortality rate cannot be ignored in the real COVID-19 data.

Model:  
S + I $\overset{\beta}{\longrightarrow}$ 2I  
I $\overset{\gamma}{\longrightarrow}$ R

$\beta$: Effective contact rate [1/min]  
$\gamma$: Recovery(+Mortality) rate [1/min]  

Ordinary Differential Equation (ODE):   
$\frac{\mathrm{d}S}{\mathrm{d}T}= - N^{-1}\beta S I$  
$\frac{\mathrm{d}I}{\mathrm{d}T}= N^{-1}\beta S I - \gamma I$  
$\frac{\mathrm{d}R}{\mathrm{d}T}= \gamma I$  

Where $N=S+I+R$ is the total population, $T$ is the elapsed time from the start date.

### Non-dimensional SIR model
To simplify the model, we will remove the units of the variables from ODE.

Set $(S, I, R) = N \times (x, y, z)$ and $(T, \beta, \gamma) = (\tau t, \tau^{-1} \rho, \tau^{-1} \sigma)$.  

This results in the ODE  
$\frac{\mathrm{d}x}{\mathrm{d}t}= - \rho x y$  
$\frac{\mathrm{d}y}{\mathrm{d}t}= \rho x y - \sigma y$  
$\frac{\mathrm{d}z}{\mathrm{d}t}= \sigma y$  

Where $N$ is the total population and $\tau$ is a coefficient ([min], is an integer to simplify).  

The range of variables and parameters:  
$0 < (x, y, z, \rho, \sigma) < 1$  
$1\leq \tau \leq 1440$  

Basic reproduction number, Non-dimentional parameter, is defined as  
$R_0 = \rho \sigma^{-1} = \beta \gamma^{-1}$  

Estimated Mean Values of $R_0$:  
$R_0$ means "the average number of secondary infections caused by an infected host" ([Infection Modeling — Part 1](https://towardsdatascience.com/infection-modeling-part-1-87e74645568a)).  
(Secondary data: [Van den Driessche, P., & Watmough, J. (2002).](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6002118))  
2.06: Zika in South America, 2015-2016  
1.51: Ebola in Guinea, 2014  
1.33: H1N1 influenza in South Africa, 2009  
3.5 : SARS in 2002-2003  
1.68: H2N2 influenza in US, 1957  
3.8 : Fall wave of 1918 Spanish influenza in Genova  
1.5 : Spring wave of 1918 Spanish influenza in Genova  

When $x=\frac{1}{R_0}$, $\frac{\mathrm{d}y}{\mathrm{d}t}=0$. This means that the max value of confirmed ($=y+z$) is $1-\frac{1}{R_0}$.

In [ ]:
train_dataset = SIR.create_dataset(
    ncov_df, population_dict["Global"] - population_dict["China"], excluded_places=[("China", None)]
)
train_start_date, train_initials, train_Tend, train_df = train_dataset
pprint([train_start_date.strftime(time_format), train_initials, train_Tend])

In [ ]:
train_df.tail()

In [ ]:
line_plot(
    train_df.set_index("T").drop("x", axis=1),
    "Training data: y(T), z(T)", math_scale=False, ylabel=""
)

**Note: We cannot convert $T$ to $t$ because $\tau$ has not been determined yet.**

### Example of non-dimensional SIR model
For example, set $R_0 = 2.5, \rho=0.2$.

In [ ]:
eg_r0, eg_rho = (2.5, 0.2)
eg_sigma = eg_rho / eg_r0
(eg_rho, eg_sigma)

In [ ]:
%%time
eg_df = simulation(SIR, train_initials, step_n=300, rho=eg_rho, sigma=eg_sigma)

In [ ]:
eg_df.tail()

In [ ]:
line_plot(
    eg_df.set_index("t"),
    title=r"SIR: $R_0$={0} ($\rho$={1}, $\sigma$={2})".format(eg_r0, eg_rho, eg_sigma),
    ylabel="",
    h=1
)

### Test of hyperparameter optimization using example data
To test the hyperparameter optimization functions defeined in this notebook, we will estimate the SIR model parameters using the example data and example $\tau=1440$ [min] and total population 1,000,000.

In [ ]:
# Set the example conditions
eg_tau = 1440
eg_start_date = ncov_df["Date"].min()
eg_total_population = 1000000
# Create dataset in the format of ncov_df
eg_ori_df = pd.DataFrame(
    {
        "Date": (eg_df["t"] * eg_tau).apply(lambda x: timedelta(minutes=x)) + eg_start_date,
        "Country": "Example",
        "Province": "Example"
    }
)
eg_ori_df["Infected"] = (eg_df["y"] * eg_total_population).astype(np.int64)
eg_ori_df["Deaths"] = (eg_df["z"] * eg_total_population * 0.02).astype(np.int64)
eg_ori_df["Recovered"] = (eg_df["z"] * eg_total_population * 0.98).astype(np.int64)
eg_ori_df.tail()

In [ ]:
# line_plot(eg_ori_df.set_index("Date")[data_cols], "Example data")

In [ ]:
# %%time
# eg_sir_estimator = Estimator(SIR, eg_ori_df, eg_total_population, places=[("Example", "Example")])
# eg_sir_dict = eg_sir_estimator.run()

In [ ]:
# eg_sir_estimator.compare_graph()

In [ ]:
"""
eg_dict = {
    "Condition": {
        "tau": eg_tau, "rho": eg_rho, "sigma": eg_sigma,
        "R0": eg_r0, "score": 0, **SIR(rho=eg_rho, sigma=eg_sigma).calc_days_dict(eg_tau)
    },
    "Estimation": eg_sir_dict
}
df = pd.DataFrame.from_dict(eg_dict, orient="index")
df
"""
None

In [ ]:
# eg_sir_estimator.predict_graph(step_n=500, name="Example area")

### Hyperparameter optimization
Using Optuna package, ($\rho, \sigma, \tau$) will be estimated by model fitting.

In [ ]:
%%time
sir_estimator = Estimator(
    SIR, ncov_df, population_dict["Except China"],
    name="Except China", excluded_places=[("China", None)]
)
sir_dict = sir_estimator.run()

In [ ]:
sir_estimator.history_df().head()

In [ ]:
sir_estimator.history_graph()

In [ ]:
pd.DataFrame.from_dict({"SIR": sir_dict}, orient="index")

In [ ]:
sir_estimator.compare_graph()

In [ ]:
sir_estimator.predict_graph(step_n=400)

## Prediction with SIR-D model
Because we can measure the number of fatal cases and recovered cases separately, we can use two variables ("Recovered" and "Deaths") instead of "Recovered + Deaths" in the mathematical model.

### What is SIR-D model?
* S: Susceptible
* I: Infected
* R: Recovered
* D: Fatal

Model:  
S + I $\overset{\beta}{\longrightarrow}$ 2I  
I $\overset{\gamma}{\longrightarrow}$ R  
I $\overset{\alpha}{\longrightarrow}$ D  

$\alpha$: Mortality rate [1/min]  
$\beta$: Effective contact rate [1/min]  
$\gamma$: Recovery rate [1/min]  

Ordinary Differential Equation (ODE):   
$\frac{\mathrm{d}S}{\mathrm{d}T}= - N^{-1}\beta S I$  
$\frac{\mathrm{d}I}{\mathrm{d}T}= N^{-1}\beta S I - (\gamma + \alpha) I$  
$\frac{\mathrm{d}R}{\mathrm{d}T}= \gamma I$  
$\frac{\mathrm{d}D}{\mathrm{d}T}= \alpha I$  

Where $N=S+I+R+D$ is the total population, $T$ is the elapsed time from the start date.

### Non-dimensional SIR-D model
Set $(S, I, R, D) = N \times (x, y, z, w)$ and $(T, \alpha, \beta, \gamma) = (\tau t, \tau^{-1} \kappa, \tau^{-1} \rho, \tau^{-1} \sigma)$.  
This results in the ODE  
$\frac{\mathrm{d}x}{\mathrm{d}t}= - \rho x y$  
$\frac{\mathrm{d}y}{\mathrm{d}t}= \rho x y - (\sigma + \kappa) y$  
$\frac{\mathrm{d}z}{\mathrm{d}t}= \sigma y$  
$\frac{\mathrm{d}w}{\mathrm{d}t}= \kappa y$  

Where $N$ is the total population and $\tau$ is a coefficient ([min], is an integer to simplify).  

The range of variables and parameters:  
$0 < (x, y, z, w, \kappa, \rho, \sigma) < 1$  
$1\leq \tau \leq 1440$

Reproduction number can be defined as  
$R_0 = \rho (\sigma + \kappa)^{-1} = \beta (\gamma + \alpha)^{-1}$

### Hyperparameter optimization
Using Optuna package, ($\kappa, \rho, \sigma, \tau$) will be estimated by model fitting.

In [ ]:
# %%time
# sird_estimator = Estimator(
#     SIRD, ncov_df, population_dict["Except China"],
#     name="Except China", excluded_places=[("China", None)],
#     tau=sir_dict["tau"], rho=sir_dict["rho"]
# )
# sird_dict = sird_estimator.run()

In [ ]:
# sird_estimator.history_graph()

In [ ]:
# pd.DataFrame.from_dict({"SIR": sir_dict, "SIR-D": sird_dict}, orient="index").fillna("-")

In [ ]:
# sird_estimator.compare_graph()

In [ ]:
# sird_estimator.predict_graph(step_n=500)

## Prediction with SIR-F model
Some cases are reported as fatal cases before clinical diagnosis of COVID-19. To consider this issue, "S + I $\to$ Fatal + I" will be added to the model.

### What is SIR-F model?
* S: Susceptible
* S$^\ast$: Confirmed and un-categorized
* I: Confirmed and categorized as I
* R: Recovered
* F: Fatal with confirmation

Measurable variables:  
Confirmed = $I+R+F$  
Recovered = $R$  
Deaths = $F$  

Model:  
S $\overset{\beta \mathrm{I}}{\longrightarrow}$ S$^\ast$ $\overset{\alpha_1}{\longrightarrow}$ F  
S $\overset{\beta \mathrm{I}}{\longrightarrow}$ S$^\ast$ $\overset{1 - \alpha_1}{\longrightarrow}$ I  
I $\overset{\gamma}{\longrightarrow}$ R  
I $\overset{\alpha_2}{\longrightarrow}$ F  

$\alpha_1$: Mortality rate of S$^\ast$ cases [-]  
$\alpha_2$: Mortality rate of I cases [1/min]  
$\beta$: Effective contact rate [1/min]  
$\gamma$: Recovery rate [1/min]  

Ordinary Differential Equation (ODE):   
$\frac{\mathrm{d}S}{\mathrm{d}T}= - N^{-1}\beta S I$  
$\frac{\mathrm{d}I}{\mathrm{d}T}= N^{-1}(1 - \alpha_1) \beta S I - (\gamma + \alpha_2) I$  
$\frac{\mathrm{d}R}{\mathrm{d}T}= \gamma I$  
$\frac{\mathrm{d}F}{\mathrm{d}T}= N^{-1}\alpha_1 \beta S I + \alpha_2 I$  

Where $N=S+I+R+F$ is the total population, $T$ is the elapsed time from the start date.

### Non-dimensional SIR-F model
Set $(S, I, R, F) = N \times (x, y, z, w)$ and $(T, \alpha_1, \alpha_2, \beta, \gamma) = (\tau t, \theta, \tau^{-1} \kappa, \tau^{-1} \rho, \tau^{-1} \sigma)$.  
This results in the ODE  
$\frac{\mathrm{d}x}{\mathrm{d}t}= - \rho x y$  
$\frac{\mathrm{d}y}{\mathrm{d}t}= \rho (1-\theta) x y - (\sigma + \kappa) y$  
$\frac{\mathrm{d}z}{\mathrm{d}t}= \sigma y$  
$\frac{\mathrm{d}w}{\mathrm{d}t}= \rho \theta x y + \kappa y$  

Where $N$ is the total population and $\tau$ is a coefficient ([min], is an integer to simplify).  

The range of variables and parameters:  
$0 < (x, y, z, w, \theta, \kappa, \rho, \sigma) < 1$  
$1 \leq \tau \leq 1440$  

Reproduction number can be defined as  
$R_0 = \rho (1 - \theta) (\sigma + \kappa)^{-1} = \beta (1 - \alpha_1) (\gamma + \alpha_2)^{-1}$

### Hyperparameter optimization
Using Optuna package, ($\theta, \kappa, \rho, \sigma, \tau$) will be estimated by model fitting.

In [ ]:
%%time
sirf_estimator = Estimator(
    SIRF, ncov_df, population_dict["Except China"],
    name="Except China", excluded_places=[("China", None)],
    tau=sir_dict["tau"]
)
sirf_dict = sirf_estimator.run()

In [ ]:
sirf_estimator.history_df().head()

In [ ]:
sirf_estimator.history_graph()

In [ ]:
pd.DataFrame.from_dict({"SIR": sir_dict, "SIR-F": sirf_dict}, orient="index").fillna("-")

In [ ]:
sirf_estimator.compare_graph()

In [ ]:
sirf_estimator.predict_graph(step_n=500)

# SIR-F model at country level

## Compare country raw data except China

In [ ]:
country_df = ncov_df.pivot_table(
    values="Infected", index="Date", columns="Country", aggfunc=sum
).fillna(0).astype(np.int64)
country_df = country_df.drop("China", axis=1)

In [ ]:
line_plot(
    country_df.T.nlargest(10, country_df.index.max()).T,
    "Infected in top 5 countries except China",
    math_scale=False
)

## Japan

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("Japan", None)])

**We will use data after 16Feb2020.**

In [ ]:
jp_start = "16Feb2020"

In [ ]:
_, jp_df = create_target_df(ncov_df, population_dict["Japan"], places=[("Japan", None)], start_date=jp_start)
line_plot(jp_df.set_index("T")[data_cols], f"Japan: without Susceptible after {jp_start}", math_scale=False)

In [ ]:
%%time
jp_estimator = Estimator(
    SIRF, ncov_df, population_dict["Japan"], name="Japan", places=[("Japan", None)],
    start_date=jp_start
)
jp_dict = jp_estimator.run()

In [ ]:
jp_estimator.compare_graph()

In [ ]:
jp_estimator.predict_graph(step_n=1200)

## South Korea

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("South Korea", None)])

**We will use data after 22Feb2020. "Infected" suddenly arised after 18Feb2020 (especially in Church of Jesus in Daegu).**

In [ ]:
sk_start = "22Feb2020"

In [ ]:
sk_start_date, sk_df = create_target_df(
    ncov_df, population_dict["South Korea"], places=[("South Korea", None)], start_date=sk_start
)
line_plot(sk_df.set_index("T")[data_cols], f"South Korea: without Susceptible after {sk_start}", math_scale=False)

In [ ]:
 %%time
 sk_estimator = Estimator(
     SIRF, ncov_df, population_dict["South Korea"], name="South Korea", places=[("South Korea", None)],
     start_date=sk_start
)
 sk_dict = sk_estimator.run()

In [ ]:
 sk_estimator.compare_graph()

In [ ]:
 sk_estimator.predict_graph(step_n=int(4000 / sk_dict["tau"]))

## Iran

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("Iran", None)])

**In Iran, the slope is slightly changed on 27Feb2020 and 07Mar2020. We will use all data because the reasons are unclear.**

In [ ]:
ir_start = ncov_df.loc[ncov_df["Country"] == "Iran", "Date"].min().strftime("%d%b%Y")
ir_start

In [ ]:
_, ir_df = create_target_df(
    ncov_df, population_dict["Iran"], places=[("Iran", None)], start_date=ir_start
)
line_plot(ir_df.set_index("T")[data_cols], f"Iran: without Susceptible after {ir_start}", math_scale=True)

In [ ]:
%%time
ir_estimator = Estimator(
    SIRF, ncov_df, population_dict["Iran"], name="Iran", places=[("Iran", None)],
    start_date=ir_start
)
ir_dict = ir_estimator.run()

In [ ]:
ir_estimator.compare_graph()

In [ ]:
ir_estimator.predict_graph(step_n=500)

ITALY

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("Italy", None)])

In [ ]:
it_start = "20Feb2020"
it_estimator = Estimator(
    SIRF, ncov_df, population_dict["Italy"], name="Italy", places=[("Italy", None)],
    start_date=it_start
)
it_dict = it_estimator.run()
it_estimator.compare_graph()
it_estimator.predict_graph(step_n=1200)

**UNITED STATES**

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("US", None)])

In [ ]:
us_start = "01Feb2020"
us_estimator = Estimator(
    SIRF, ncov_df, population_dict["US"], name="US", places=[("US", None)],
    start_date=us_start
)
us_dict = us_estimator.run()
us_estimator.compare_graph()
us_estimator.predict_graph(step_n=1200)

In [ ]:
first_model, info_dict, param_dict = us_estimator.info()
predicter = Predicter(**info_dict)
predicter.add(first_model, end_day_n=350, count_from_last=False, **param_dict)
predicter.restore_graph(drop_cols=None)

## Parameter comparsion of the countries

In [ ]:
_dict = {
    "Except China": sirf_dict,
    "Japan": jp_dict,
    "South Korea": sk_dict,
    "Iran": ir_dict,
    "Italy": it_dict,
    "US": us_dict,
}
pd.DataFrame.from_dict(_dict, orient="index")

# The number of exposed cases and waiting cases
The number of exposed cases in latent period (E) and wating cases for confirmation (W) are un-measurable variables, but key variables as well as S, I, R, F. If E and W are large, outbreak will occur in the near future. Let's replace S $\overset{\beta \mathrm{I}}{\longrightarrow}$ S$^\ast$ with S $\overset{\beta_1 \mathrm{(W+I)}}{\longrightarrow}$ E $\overset{\beta_2}{\longrightarrow}$ W $\overset{\beta_3}{\longrightarrow}$ S$^\ast$ because W also has infectivity.

Note:  
W and some rules were added to explain COVID-19 dataset, but this is like-SEIR model.  
To study general SEIR-model, please refer to PDF material in [Introduction to SEIR model Models](http://indico.ictp.it/event/7960/session/3/contribution/19/material/slides/).

## What is SEWIR-F model?
* S: Susceptible
* <u>E: Exposed and in latent period</u>
* <u>W: Waiting cases for confirmation (with infectivity)</u>
* S$^\ast$: Confirmed and un-categorized
* I: Confirmed and categorized as I
* R: Recovered
* F: Fatal with confirmation

Measurable variables:  
Total population - Confirmed = $S+E+W+S^\ast$  
Confirmed = $I+R+F$  
Recovered = $R$  
Deaths = $F$  

Model:  
S $\overset{\beta_1 \mathrm{(W+I)}}{\longrightarrow}$ E $\overset{\beta_2}{\longrightarrow}$ W $\overset{\beta_3}{\longrightarrow}$ S$^\ast$ $\overset{\alpha_1}{\longrightarrow}$ F  
S $\overset{\beta_1 \mathrm{(W+I)}}{\longrightarrow}$ E $\overset{\beta_2}{\longrightarrow}$ W $\overset{\beta_3}{\longrightarrow}$ S$^\ast$ $\overset{1 - \alpha_1}{\longrightarrow}$ I  
I $\overset{\gamma}{\longrightarrow}$ R  
I $\overset{\alpha_2}{\longrightarrow}$ F  

$\alpha_1$: Mortality rate of S$^\ast$ cases [-]  
$\alpha_2$: Mortality rate of I cases [1/min]  
$\beta_1$: <u>Exposure rate (the number of encounter with the virus in a minute)</u> [1/min]  
$\beta_2$: <u>Inverse of latent period</u> [1/min]  
$\beta_3$: <u>Inverse of waiting time for confirmation</u> [1/min]  
$\gamma$: Recovery rate [1/min]  

Ordinary Differential Equation (ODE):   
$\frac{\mathrm{d}S}{\mathrm{d}T}= - N^{-1}\beta_1 S (W + I)$  
$\frac{\mathrm{d}E}{\mathrm{d}T}= N^{-1}\beta_1 S (W + I) - \beta_2 E$  
$\frac{\mathrm{d}W}{\mathrm{d}T}= \beta_2 E - \beta_3 W$  
$\frac{\mathrm{d}I}{\mathrm{d}T}= (1 - \alpha_1)\beta_3 W - (\gamma + \alpha_2) I$  
$\frac{\mathrm{d}R}{\mathrm{d}T}= \gamma I$  
$\frac{\mathrm{d}F}{\mathrm{d}T}= N^{-1}\alpha_1 \beta_2 E + \alpha_2 I$  

Where $N=S+E+W+I+R+F$ is the total population, $T$ is the elapsed time from the start date.

## Non-dimensional SEIR-F model
Set $(S, E, W, I, R, F) = N \times (x_1, x_2, x_3, y, z, w)$, $(T, \alpha_1) = (\tau t, \theta)$ and $(\alpha_2, \beta_i, \gamma) = \tau^{-1} \times (\kappa, \rho_i, \sigma)$.  
This results in the ODE  
$\frac{\mathrm{d}x_1}{\mathrm{d}t}= - \rho_1 x_1 (x_3 + y)$  
$\frac{\mathrm{d}x_2}{\mathrm{d}t}= \rho_1 x_1 (x_3 + y) - \rho_2 x_2$  
$\frac{\mathrm{d}x_3}{\mathrm{d}t}= \rho_2 x_2 - \rho_3 x_3$  
$\frac{\mathrm{d}y}{\mathrm{d}t}= (1-\theta) \rho_3 x_3 - (\sigma + \kappa) y$  
$\frac{\mathrm{d}z}{\mathrm{d}t}= \sigma y$  
$\frac{\mathrm{d}w}{\mathrm{d}t}= \theta \rho_2 x_2 + \kappa y$  

Where $N$ is the total population and $\tau$ is a coefficient ([min], is an integer to simplify).  

The range of variables and parameters:  
$0 < (x_i, y, z, w, \theta, \kappa, \rho_i, \sigma) < 1$  
$1 \leq \tau \leq 1440$  

Reproduction number can be defined as  
$R_0 = \rho_1 (1-\theta) (\sigma + \kappa)^{-1}$

## Calculate $\rho_2$ and $\rho_3$
To estimate $\rho_2=\tau \beta_2$ and $\rho_3=\tau \beta_3$, we first calculate median value of latent period $\overline{L_E}$ and waiting time for confirmation $\overline{L_W}$ using linelist. We assume that patients start to have infectivity from onset dates. This means latent period is equal to incubation period.

$\beta_2$: Inverse of latent period [1/min]  
$\beta_3$: Inverse of waiting time for confirmation [1/min]  

In [ ]:
period_df = select_area(linelist_df, excluded_places=[("China", None)], group=None)
period_df = period_df.loc[:, ["Exposed_date", "Onset_date", "Confirmed_date"]]
period_df["Latent [min]"] = (period_df["Onset_date"] - period_df["Exposed_date"]).dt.total_seconds() / 60
period_df["Waiting [min]"] = (period_df["Confirmed_date"] - period_df["Onset_date"]).dt.total_seconds() / 60
period_df["Latent [day]"] = period_df["Latent [min]"] / 60 / 24
period_df["Waiting [day]"] = period_df["Waiting [min]"] / 60 / 24
period_df["Latent + Waiting [day]"] = period_df["Latent [day]"] + period_df["Waiting [day]"]
period_df.dropna(axis=0).tail()

In [ ]:
cols = ["Latent [day]", "Waiting [day]", "Latent + Waiting [day]"]
period_df[cols].plot.kde()
plt.title("Kernel density estimation of latent period and waiting time for confirmation [day]")
plt.show()
period_df[cols].describe().T

In [ ]:
latent_period = period_df["Latent [min]"].median()
waiting_time = period_df["Waiting [min]"].median()
tau = sirf_estimator.info()[1]["tau"]
rho2, rho3 = tau / latent_period, tau / waiting_time
(rho2, rho3)

In [ ]:
# We will use in scenario analysis section
latent_waiting_day = period_df["Latent + Waiting [day]"].median()
latent_waiting_day

## Estimate $\rho_1$
We wil estimate $\rho_1$ using the model and dataset.

In [ ]:
param_dict = sirf_estimator.info()[2]
param_dict.pop("rho")
param_dict["tau"] = tau
param_dict["rho2"] = rho2
param_dict["rho3"] = rho3
param_dict

In [ ]:
%%time
sewirf_estimator = Estimator(
    SEWIRF, ncov_df, population_dict["Except China"],
    name="Except China", excluded_places=[("China", None)],
    **param_dict
)
sewirf_dict = sewirf_estimator.run()

In [ ]:
pd.DataFrame.from_dict({"SIR": sir_dict, "SIR-F": sirf_dict, "SEWIR-F": sewirf_dict}, orient="index").fillna("-")

In [ ]:
sewirf_estimator.compare_graph()

## Prediction with SEWIR-F model

In [ ]:
first_model, info_dict, param_dict = sewirf_estimator.info()
info_dict["name"] = "Except China"
pd.DataFrame.from_dict({"No actions": param_dict}, orient="index")

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=500, count_from_last=False, **param_dict)
predicter.restore_graph(drop_cols=None)

# Scenario analysis
To figure out what to do for minimizing the damage, we will perform scenario analysis in this section. 
* Suggest real factors of the parameters of the math model
* Collect/predict records of the real factors
* Calcurate the impact of actions on the parameters
* Predict the future in the assumption that new actions are taken since today
* Figure out what to do right now

## Real factors of effective contact rate $\beta_1$
Please reconsider S $\overset{\beta_1 \mathrm{(W+I)}}{\longrightarrow}$ E formula. Susceptible persons may contact with currently infected patients, and susceptible persons will be infected with COVID-19. The formura can be replaced with  
S$_\mathrm{q}$ $\overset{g_{s}}{\Longleftrightarrow}$ S$_{\mathrm{g}}$ $\overset{f_1}{\longrightarrow}$ E$^\ast$ $\overset{e^{-h_2}}{\longrightarrow}$ E  
I$_\mathrm{q}$ $\overset{g_i}{\Longleftrightarrow}$ I$_{\mathrm{g}}$  
I$_\mathrm{q}$ $\overset{q}{\longrightarrow}$ I$_{\hat{\mathrm{q}}}$  
E$^\ast$ $\overset{1-e^{-h_2}}{\longrightarrow}$ R$^\ast$

$\Longleftrightarrow$ (as substitute for $\longrightarrow$ with $\longleftarrow$) means that right side can be return to the left side.  
S$_\mathrm{q}$: Susceptible persons in quarantine <!--Susceptible in the strict sense-->  
S$_\mathrm{g}$: Susceptible persons with family members or friends etc.  
I$_\mathrm{q}$: Currently infected patients in quarantine  
I$_\mathrm{g}$: Currently infected patients with family members or friends etc.  
I$_\hat{\mathrm{q}}$: Currently infected patients who was hospitalized  
E$^\ast$: Just after being exposed to the virus  
R$^\ast$: Being exposed to the virus, fighted with the virus, recovered and immuned without confirmation  

$f_1 = vI_{\mathrm{g}}(1-m)^2(1-w_e)^{w_n}e^{-h_1}sc$ [-] 

Real factors:  
$g_s$: The number of days in <u>a week</u> susceptible persons go out [day]  
$g_i$: The number of days in <u>a week</u> currently infected (confirmed) but un-quarantined persons go out [day]  
$q$: Quarantine rate of currently infected (confirmed) patients [-]  
$v$: Probability of virus existance in a droplet [-]  
$m$: Rate of persons wearing masks effectively (depends logistically on supply of masks) [-]  
$w_e$: Virus reduction effect of washing hands [-]  
$w_n$: The number of times people washes their hands before touching their own faces after go out [-]  
$h_1$: Health condition (active rate of cellular immunity factors) of susceptible and contacted persons [-]  
$h_2$: Health condition (active rate of humoral immunity factors) of susceptible and contacted persons [-]  
$c$: The number of contacts between susceptible persons and patients while on the go in a minute (depends on population density) [1/min]  
$\delta$:The product of unknown real factors [-]  

The parameter in the math model:  
$\beta_1 = \cfrac{g_s g_i (1-q) v (1-m)^2 (1-w_e)^{w_n} e^{-(h_{1}+h_{2})} c \delta}{49}$ [1/min]


In [ ]:
# Value of beta before actions are taken
beta1_before = param_dict["rho1"] / info_dict["tau"]
beta1_before

### $g_s$ value before actions are taken
$g_s$: The number of days in <u>a week</u>, susceptible persons go out [day]  

We can calculate weighted average of days with age composion of population.

In [ ]:
df = out_df.copy()
df["Portion"] = df["Except China"]
ec_out_df = df.drop(population_dict.keys(), axis=1)
ec_out_df

In [ ]:
gs_before = (ec_out_df[["School", "Office", "Others"]].sum(axis=1) * ec_out_df["Portion"]).sum()
gs_before

### $g_s$ value AFTER actions are taken
If all schools and offices will be closed, $g_s$ can be reduced. People will go out one day for other reasons instead of going to school/office.

In [ ]:
df = ec_out_df.copy()
df["School"] = 0
df["Office"] = 0
df.loc[df.index[1:9], "Others"] += 1
gs_after = (df[["School", "Office", "Others"]].sum(axis=1) * df["Portion"]).sum()
gs_after

## Impact of actions on $\beta_1$
Actions to take since today:  
All schools and offices will be closed.  

In [ ]:
beta1_after = beta1_before * (gs_after / gs_before)
beta1_after / beta1_before

## Predict the future: with actions since today
There is a delay between the time point of starting actions and that of appearing the effect. Because I is the main variable, the length of delay can be estimated as sum of latent period and waiting time for confirmation. This value [day] was calculated in "The number of exposed cases and waiting cases" section.

In [ ]:
latent_waiting_day

In [ ]:
first_model, info_dict, param_dict = sewirf_estimator.info()
info_dict["name"] = "Except China"
changed_param_dict = param_dict.copy()
changed_param_dict["rho1"] = param_dict["rho1"] * beta1_after / beta1_before
df = pd.DataFrame.from_dict(
    {"No actions": param_dict, "With actions": changed_param_dict},
    orient="index"
)
df = df.loc[:, ["rho1", "rho2", "rho3", "theta", "kappa", "sigma"]]
df["R0"] = df.apply(lambda x: first_model(**x.to_dict()).calc_r0(), axis=1)
df["tau"] = info_dict["tau"]
df

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=latent_waiting_day, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=latent_waiting_day, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=800, count_from_last=False, **changed_param_dict)
predicter.restore_graph(drop_cols=None)

The actions result in:  
Total number of confirmed cases was decreased.  
Peak point of infected cases was delayed.   
We need to fight with the virus for longer period.

## Real factors of recovery rate $\gamma$ and mortality rate $\alpha_2$
Here, let's reconsider I $\overset{\gamma}{\longrightarrow}$ R and I $\overset{\alpha_2}{\longrightarrow}$ F.  
Because balance of immunity (+effect of treatments) and virulence determines whether patients can recover or not, the formulas can be replaced with  

I $\overset{\bar{h}}{\longrightarrow}$ I$^\star$ $\overset{\bar{s}}{\longrightarrow}$ F$^\star$ $\overset{L^{-1}}{\longrightarrow}$ F  
I $\overset{f_2}{\longrightarrow}$ R$^\star$ $\overset{l^{-1}}{\longrightarrow}$ R  

I$^\star$: Confirmed cases whose immune systems did not overcome virus multiplication, and <u>without</u> severe events  
F$^\star$: Confirmed cases whose immune systems did not overcome virus multiplication, and <u>with</u> severe events  
R$^\star$: Confirmed cases whose immune systems overcame virus multiplication or comfirmed cases whose severe events can be stopped

Where $f_2 = 1 - \bar{h}\ \bar{s}$  

$\bar{h}$: Rate of I whose immune systems does NOT overcame virus multiplication [-]  
$\bar{s}$: Rate of I$^\star$ who have severe events, including respiratory failure  [-]  
$L_i$: Inverse of F$^\star$'s mortality rate for people $i$ years old [min]  
$l_i$: Inverse of R$^\star$'s mortality rate for people $i$ years old [min]  
$P_i$: The number of people $i$ years old [-]  
$N$: Total population  

\begin{align*}
& \alpha_2 = \cfrac{\bar{h}\ \bar{s}}{N} \sum_{n=0}^{\infty}\cfrac{P_{i}}{L_i} \\
    & \gamma = \cfrac{1 - \bar{h}\ \bar{s}}{N} \sum_{n=0}^{\infty}\cfrac{P_{i}}{l_i} \\
\end{align*}

## $\bar{h}$ and $\bar{s}$ value before actions are taken
We assume that $\bar{h}=0.5$ and $\bar{s}=0.5$. **(Yes, we need remove this assumtions later!)**  
**(Using population distribution data and case reports, $\bar{h}\ \bar{s}$ and $1 - \bar{h}\ \bar{s}$ can be calculated.)**

In [ ]:
gamma_before = param_dict["sigma"] / info_dict["tau"]
alpha2_before = param_dict["kappa"] / info_dict["tau"]
(gamma_before, alpha2_before)

In [ ]:
h_bar_before, s_bar_before = 0.5, 0.5

## $\bar{h}$ and $\bar{s}$ value AFTER actions are taken
Assumtions of new medicines:  
"Protease inhibitor" inhibits virus multiplication. This will reduce $\bar{h}$

In [ ]:
h_bar_after = 0.25
s_bar_after = s_bar_before

## Impact on $\gamma$ and $\alpha_2$
Actions to take:  
New Protein inhibitor medicine was introduced.

In [ ]:
gamma_after = gamma_before * (1 - h_bar_after * s_bar_after) / (1 - h_bar_before * s_bar_before)
gamma_after

In [ ]:
alpha2_after = alpha2_before * (h_bar_after * s_bar_after) / (h_bar_before * s_bar_before)
alpha2_after

## Predict the future: with actions from the next month

In [ ]:
first_model, info_dict, param_dict = sewirf_estimator.info()
info_dict["name"] = "Except China"
changed_param_dict = param_dict.copy()
changed_param_dict["sigma"] = param_dict["sigma"] * gamma_after / gamma_before
changed_param_dict["kappa"] = param_dict["kappa"] * alpha2_after / alpha2_before
df = pd.DataFrame.from_dict(
    {"No actions": param_dict, "With actions": changed_param_dict},
    orient="index"
)
df = df.loc[:, ["rho1", "rho2", "rho3", "theta", "kappa", "sigma"]]
df["R0"] = df.apply(lambda x: first_model(**x.to_dict()).calc_r0(), axis=1)
df["tau"] = info_dict["tau"]
df

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=500, count_from_last=False, **changed_param_dict)
predicter.restore_graph(drop_cols=None)

The actions result in:  
Total numbers of confirmed/deaths cases were decreased.

## If 10,000,000/day are vaccinated (SIR-FV model) since today
We will predict the numbers of cases in the assumption that 10,000,000 persons will be vacctinated in one day until there are susceptible people.  
$\frac{\mathrm{d}x}{\mathrm{d}t}= - \rho x y - \omega$  
$\frac{\mathrm{d}y}{\mathrm{d}t}= \rho (1-\theta) x y - (\sigma + \kappa) y$  
$\frac{\mathrm{d}z}{\mathrm{d}t}= \sigma y$  
$\frac{\mathrm{d}w}{\mathrm{d}t}= \rho \theta x y + \kappa y$  
Where $\omega_{(x>0)}=\frac{10,000,000}{N}$ and $N$ is the total population.

Reproduction number can be defined as  
$R_0 = \rho (1 - \theta) (\sigma + \kappa)^{-1}$

In [ ]:
first_model, info_dict, param_dict = sirf_estimator.info()
changed_param_dict = param_dict.copy()
changed_param_dict["n"] = population_dict["Global"] - population_dict["China"]
changed_param_dict["v_per_day"] = 10000000
predicter = Predicter(**info_dict)
predicter.add(SIRF, end_day_n=None, count_from_last=False, **param_dict)
predicter.add(SIRFV, end_day_n=700, count_from_last=False, **changed_param_dict)
predicter.restore_graph(drop_cols=None)

# Scenario analysis in Italy
In this section, we will perform scenario analysis using data in Italy.

## Trend analysis

In [ ]:
show_trend(ncov_df, variable="Confirmed", places=[("Italy", None)])

**We will use the records from 23Feb2020.**

In [ ]:
it_start = "23Feb2020"

In [ ]:
_, it_df = create_target_df(
    ncov_df, population_dict["Italy"], places=[("Italy", None)], start_date=it_start
)
line_plot(it_df.set_index("T")[data_cols], f"US: without Susceptible after {it_start}", math_scale=False)

## Estimate parameters of SIR-F model

In [ ]:
%%time
it_sirf_estimator = Estimator(
    SIRF, ncov_df, population_dict["Italy"], name="Italy", places=[("Italy", None)],
    start_date=it_start
)
it_sirf_dict = it_sirf_estimator.run()

In [ ]:
it_sirf_estimator.compare_graph()

## Calculate $\rho_2$ and $\rho_3$
To estimate $\rho_2=\tau \beta_2$ and $\rho_3=\tau \beta_3$, we first calculate median value of latent period $\overline{L_E}$ and waiting time for confirmation $\overline{L_W}$ using linelist.  

$\beta_2$: Inverse of latent period [1/min]  
$\beta_3$: Inverse of waiting time for confirmation [1/min]  

In [ ]:
select_area(linelist_df, places=[("Italy", None)], group=None)

**Because the number of records in Italy is small, we will not use linelist data.**

## Estimate parameters of SEWIR-F model

In [ ]:
_, info_dict, param_dict = it_sirf_estimator.info()
param_dict.pop("rho")
param_dict["tau"] = info_dict["tau"]
param_dict

In [ ]:
%%time
it_estimator = Estimator(
    SEWIRF, ncov_df, population_dict["Italy"], name="Italy", places=[("Italy", None)],
    start_date=it_start,
    **param_dict
)
it_dict = it_estimator.run(700)

In [ ]:
pd.DataFrame.from_dict({"SIR-F": it_sirf_dict, "SEWIR-F": it_dict}, orient="index").fillna("-")

In [ ]:
it_estimator.compare_graph()

## Prediction with SEWIR-F model

In [ ]:
first_model, info_dict, param_dict = it_estimator.info()
pd.DataFrame.from_dict({"No actions": param_dict}, orient="index")

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(first_model, end_day_n=None, count_from_last=False, **param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(first_model, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(first_model, end_day_n=30, count_from_last=True, **param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(first_model, end_day_n=300, count_from_last=False, **param_dict)
predicter.restore_graph(drop_cols=None)

## Effect of lockdown
Acording to first report of [COVID-19 Mobility Monitoring project](https://covid19mm.github.io/) on 13Mar2020, the government of Italy declared a national lockdown on 09Mar2020 and all peole are asked to remain home. This resulted in average reduction of potential encounters of 19% during week 3 (from 07Mar2020 to 10Mar2020).  
**Here, we will predict the effect of lockdown on 13Mar2020 with assumtion that the effect will be shown on 01Apr2020.**

### How many days we have until the day the effect will be shown?

In [ ]:
lockdown_delay = int((datetime.strptime("01Apr2020", "%d%b%Y") - datetime.today()).total_seconds() / 60 / 60 / 24)
lockdown_delay

## Real factors of $\beta_1$

The parameter in the math model:  
$\rho_1 = \tau \beta_1$  
$\beta_1 = \cfrac{g_s g_i (1-q) v (1-m)^2 (1-w_e)^{w_n} e^{-(h_{1}+h_{2})} c \delta}{49}$ [1/min]

Real factors:  
$g_s$: <u>The number of days in a week susceptible persons go out</u> [day]  
$g_i$: The number of days in a week currently infected (confirmed) but un-quarantined persons go out [day]  
$q$: Quarantine rate of currently infected (confirmed) patients [-]  
$v$: Probability of virus existance in a droplet [-]  
$m$: Rate of persons wearing masks effectively (depends logistically on supply of masks) [-]  
$w_e$: Virus reduction effect of washing hands [-]  
$w_n$: The number of times people washes their hands before touching their own faces after go out [-]  
$h_1$: Health condition (active rate of cellular immunity factors) of susceptible and contacted persons [-]  
$h_2$: Health condition (active rate of humoral immunity factors) of susceptible and contacted persons [-]  
$c$: <u>The number of contacts between susceptible persons and patients while on the go in a minute (depends on population density)</u> [1/min]  
$\delta$:The product of unknown real factors [-]  



### Value of real factors of $\beta_1$ before/after the national lockdown
A national lockdown will effect on $g_s$ and $c$.

### $g_s$ before the lockdown
We will estimate average number peple go out using @marcoferrante estimation table and population pyramid data.
It is necessary to replace the population pyramid data for Italy because the situation is different from the average data.

In [ ]:
df = out_df.copy()
df["Portion"] = df["Italy"]
it_out_df = df.drop(population_dict.keys(), axis=1)
it_out_df

In [ ]:
gs_before = (it_out_df[["School", "Office", "Others"]].sum(axis=1) * it_out_df["Portion"]).sum()
gs_before

Estimation of $g_s$ after the lockdown is here.  
This is a rough estimate, and we need detailed information.

In [ ]:
df = it_out_df.copy()
df["School"] = 0
df.loc[df["Office"] > 0, "Office"] = 0.5
df.loc[df["Others"] > 0, "Others"] = 0.6
it_out_after_df = df.copy()
it_out_after_df

In [ ]:
gs_after = (it_out_after_df[["School", "Office", "Others"]].sum(axis=1) * it_out_after_df["Portion"]).sum()
gs_after

Acccoring the report, we assume average reduction of potential encounters of 19%.

In [ ]:
c_before, c_after = 1.0, 0.81

### Effect of the lockdown on $\rho_1$

In [ ]:
rho1_before = param_dict["rho1"]
rho1_after = rho1_before * (gs_after / gs_before) * (c_after / c_before)
(rho1_before, rho1_after)

### Predict the future with the national lockdown

In [ ]:
changed_param_dict = param_dict.copy()
changed_param_dict["rho1"] = rho1_after
df = pd.DataFrame.from_dict(
    {"No actions": param_dict, "With actions": changed_param_dict},
    orient="index"
)
df["R0"] = df.apply(lambda x: first_model(**x.to_dict()).calc_r0(), axis=1)
df["tau"] = info_dict["tau"]
df

In a week,

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=lockdown_delay, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=7, count_from_last=True, **changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

In 30 days,

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=lockdown_delay, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"])

Values are here.

In [ ]:
df = predicter.restore_df()
df.loc[datetime.today():, :].head(30).style.background_gradient(axis=0)

In the long-term,

In [ ]:
predicter.restore_graph(drop_cols=["Susceptible"])

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=lockdown_delay, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=1000, count_from_last=False, **changed_param_dict)
predicter.restore_graph(drop_cols=None)

Time point of peak may be delayed.

## Effect of expected new medicines (Favipiravir, AVIGAN)
New medicines are necessary so that patients can recover more quicky from the disease. Drug repositioning strategy (i.e.finding effective candidates from library of existing drugs of different diseases) is used to develop the medicines of COVID-19. For example, Favipiravir (AVIGAN) is a candidate. Certainly, this medicine may lead many serious adverse reactions and it cannot be provided to expectant mothers [KEGG database AVIGAN](https://www.kegg.jp/medicus-bin/japic_med?japic_code=00066852) (Sorry, this is written in Japanese). However, it may help to save many thousand lives.  

**We do not have information about its criteria to administrate and medicinal effect on COVID-19. We assume that fatal risk $\bar{h}\ \bar{s}$ will be halved (0.50 $\to$ 0.25) from 01May2020.**
(The values may be different for each age group, but we assume they are constant now.)

Where  
\begin{align*}
& \kappa \tau^{-1} = \alpha_2 = \cfrac{\bar{h}\ \bar{s}}{N} \sum_{n=0}^{\infty}\cfrac{P_{i}}{L_i} \\
& \sigma \tau^{-1} = \gamma = \cfrac{1 - \bar{h}\ \bar{s}}{N} \sum_{n=0}^{\infty}\cfrac{P_{i}}{l_i} \\
\end{align*}

$\bar{h}$: Rate of I whose immune systems does NOT overcame virus multiplication [-]  
$\bar{s}$: Rate of I$^\star$ who have severe events, including respiratory failure  [-]  
$L_i$: Inverse of F$^\star$'s mortality rate for people $i$ years old [min]  
$l_i$: Inverse of R$^\star$'s mortality rate for people $i$ years old [min]  
$P_i$: The number of people $i$ years old [-]  
$N$: Total population  



In [ ]:
med_delay = int((datetime.strptime("01May2020", "%d%b%Y") - datetime.today()).total_seconds() / 60 / 60 / 24)
med_delay

In [ ]:
med_changed_param_dict = changed_param_dict.copy()
med_changed_param_dict["kappa"] = changed_param_dict["kappa"] * 0.25 / 0.50
med_changed_param_dict["sigma"] = changed_param_dict["sigma"] * (1 - 0.25) / (1 - 0.50)
_dict = {
    "No actions": param_dict,
    "National lockdown": changed_param_dict,
    "Lockdown + Medicine": med_changed_param_dict
}
df = pd.DataFrame.from_dict(_dict, orient="index")
df["R0"] = df.apply(lambda x: first_model(**x.to_dict()).calc_r0(), axis=1)
df["tau"] = info_dict["tau"]
df

In three months,

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=lockdown_delay, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **changed_param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **med_changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"], y_integer=True)

Values are here,

In [ ]:
df = predicter.restore_df()
df.loc[datetime.today():, :].head(90).style.background_gradient(axis=0)

In [ ]:
predicter = Predicter(**info_dict)
predicter.add(SEWIRF, end_day_n=None, count_from_last=False, vline=False, **param_dict)
predicter.add(SEWIRF, end_day_n=lockdown_delay, count_from_last=True, **param_dict)
predicter.add(SEWIRF, end_day_n=30, count_from_last=True, **changed_param_dict)
predicter.add(SEWIRF, end_day_n=1000, count_from_last=False, **med_changed_param_dict)
predicter.restore_graph(drop_cols=["Susceptible"], y_integer=True)

# Remarks and To-do list
Thank you for reading!  
Lisphilar from Japan

## Remarks

Non-dimentional SIR/SIR-D/SIR-F model explained the total data except China. Predicted values with the parameters estimated by model fitting is under the assumption that no new actions will be taken. If no actions will be taken, almost all peaple may be infected by this disease...


If trend changes, including the effect of measures, are observed in "Trend analysis" section, we will need to replace the model to apply to the dataset after the change points. It is difficult to add another varible/parameter to the model because we can measure only 4 variables (confirmed, recovered, deaths, total population) now. To improve the accuracy, ODEs should be replaced.

With the models, we predicted the effect of actions on the numbers of cases. New medicines and vaccines are expected, but, in initial state, we must make an effort to minimize effective contact rate $\rho$ by quarantining/wearing of masks. This effort will reduce the number of infected cases directory.

## To-do list

* Find out the root cause of unstable accuracy of estimation and improve Estimater class
* Increase speed of the functions and classes. Estimater.run(), which uses Optuna and scipy.integrate.solve_ivp, is a key method, but time-consuming one.
* Reconsider the ODEs of SIR-F model using new records
* Find out the change points using case reports and news information, and feed-back to the model.
* Discuss $R_0$ using the data of the other infectious diseases
* Calculate the impact of preventive actions on the parameters of math models and find out effective preventive actions.